In [ ]:
import getpass
import sys
import os
import os.path
home = os.environ.get("HOME")
sys.path.append(f"{home}/bioinformatics/lib")
sys.path.append(f"{home}/bioinformatics/notebooks/analysis/chronic_pain/publishable/src/lib")

import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
from scipy.sparse import issparse
import anndata
from anndata import AnnData
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from collections import defaultdict, OrderedDict
import gzip
import csv
import re
import io
import logging
import warnings
import subprocess
from preprocessing import *

FORMAT = '%(asctime)-15s %(message)s'
logging.basicConfig(format=FORMAT)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 50)
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=120, color_map="cividis")

In [ ]:
dataset = "GSE249746"
species = "Human"

In [ ]:
dirname = os.getcwd()
source = os.path.abspath(os.path.join(dirname, f'../../build/datasets/{dataset}'))
fns = os.listdir(os.path.join(source, 'downloads'))
print(source)
fns

In [ ]:
fn = 'GSE249746_Expression_matrix_raw_counts.csv.gz'
df = pd.read_csv(os.path.join(source, 'downloads', fn), header=0, compression='gzip', sep=',', quotechar='"')
df = df.set_index('Unnamed: 0')
df

In [ ]:
adata = AnnData(np.array(df.T), dtype=float)
adata.obs_names = list(df.columns)
adata.var_names = list(df.index)
adata

In [ ]:
# curated metadata from GEO series matrix file

meta = """
sample_id	"N2-RL5-1"	"N2-RL5-2"	"N2-RL5-3"	"N2-RL5-4"	"N2-RL5-5"	"N2-RL5-6"	"N2-RL5-7"	"N2-RL5-8"	"N2-RL5-9"	"N2-RL5-10"	"N2-RL5-11"	"N2-RL5-12"	"N2-RL5-13"	"N2-RL5-14"	"N2-RL5-15"	"N2-RL5-16"	"N2-RL5-17"	"N2-RL5-18"	"N2-RL5-19"	"N2-RL5-20"	"N2-RL5-21"	"N2-RL5-22"	"N2-RL5-23"	"N2-RL5-24"	"N2-RL5-25"	"N2-RL5-26"	"N2-RL5-28"	"N2-RL5-29"	"N2-RL5-30"	"N2-RL5-31"	"N2-RL5-32"	"N2-RL5-33"	"N2-RL5-34"	"N2-RL5-35"	"N2-RL5-36"	"N2-RL5-37"	"N2-RL5-38"	"N2-RL5-39"	"N2-RL5-40"	"N2-RL5-41"	"N2-RL5-42"	"N2-RL5-43"	"N2-RL5-44"	"N2-RL5-45"	"N2-RL5-47"	"N2-RL5-48"	"N2-RL5-49"	"N2-RL5-50"	"N2-RL5-51"	"N2-RL5-52"	"N2-RL5-53"	"N2-RL5-54"	"N2-RL5-55"	"N2-RL5-56"	"N2-RL5-57"	"N2-RL5-58"	"N2-RL5-59"	"N2-RL5-60"	"N2-RL5-61"	"N2-RL5-62"	"N2-RL5-63"	"N2-RL5-64"	"N2-RL5-65"	"N2-RL5-66"	"N2-RL5-67"	"N2-RL5-68"	"N2-RL5-69"	"N2-RL5-70"	"N2-RL5-71"	"N2-RL5-73"	"N2-RL5-74"	"N2-RL5-75"	"N2-RL5-76"	"N2-RL5-77"	"N2-RL5-78"	"N2-RL5-79"	"N2-RL5-80"	"N2-RL5-81"	"N2-RL5-82"	"N2-RL5-83"	"N2-RL5-84"	"N2-RL5-85"	"N2-RL5-86"	"N2-RL5-87"	"N2-RL5-88"	"N2-RL5-89"	"N2-RL5-90"	"N2-RL5-91"	"N2-RL5-92"	"N2-RL5-93"	"N2-RL5-94"	"N2-RL5-95"	"N2-RL5-96"	"N2-RT12-1"	"N2-RT12-2"	"N2-RT12-3"	"N2-RT12-5"	"N2-RT12-6"	"N2-RT12-7"	"N2-RT12-8"	"N2-RT12-9"	"N2-RT12-10"	"N2-RT12-11"	"N2-RT12-12"	"N2-RT12-13"	"N2-RT12-14"	"N2-RT12-15"	"N2-RT12-16"	"N2-RT12-17"	"N2-RT12-20"	"N2-RT12-21"	"N2-RT12-22"	"N2-RT12-23"	"N2-RT12-24"	"N2-RT12-25"	"N2-RT12-26"	"N2-RT12-27"	"N2-RT12-28"	"N2-RT12-29"	"N2-RT12-30"	"N2-RT12-31"	"N2-RT12-32"	"N2-RT12-33"	"N2-RT12-34"	"N2-RT12-35"	"N2-RT12-36"	"N2-RT12-38"	"N2-RT12-39"	"N2-RT12-40"	"N2-RT12-41"	"N2-RT12-42"	"N2-RT12-43"	"N2-RT12-44"	"N2-RT12-45"	"N2-RT12-46"	"N2-RT12-47"	"N2-RT12-48"	"N2-RT12-49"	"N2-RT12-50"	"N2-RT12-51"	"N2-RT12-52"	"N2-RT12-53"	"N2-RT12-54"	"N2-RT12-55"	"N2-RT12-56"	"N2-RT12-57"	"N2-RT12-58"	"N2-RT12-59"	"N2-RT12-60"	"N2-RT12-61"	"N2-RT12-62"	"N2-RT12-63"	"N2-RT12-64"	"N2-RT12-65"	"N2-RT12-66"	"N2-RT12-67"	"N2-RT12-68"	"N2-RT12-69"	"N2-RT12-70"	"N2-RT12-71"	"N2-RT12-72"	"N2-RT12-73"	"N2-RT12-74"	"N2-RT12-75"	"N2-RT12-76"	"N2-RT12-77"	"N2-RT12-78"	"N2-RT12-79"	"N2-RT12-80"	"N2-RT12-81"	"N2-RT12-82"	"N2-RT12-83"	"N2-RT12-84"	"N2-RT12-85"	"N2-RT12-86"	"N2-RT12-87"	"N2-RT12-88"	"N2-RT12-89"	"N2-RT12-90"	"N2-RT12-91"	"N2-RT12-92"	"N2-RT12-93"	"N2-RT12-94"	"N2-RT12-95"	"N2-RT12-96"	"N3-RL2-1"	"N3-RL2-2"	"N3-RL2-3"	"N3-RL2-4"	"N3-RL2-5"	"N3-RL2-6"	"N3-RL2-7"	"N3-RL2-8"	"N3-RL2-9"	"N3-RL2-10"	"N3-RL2-11"	"N3-RL2-12"	"N3-RL2-13"	"N3-RL2-14"	"N3-RL2-15"	"N3-RL2-16"	"N3-RL2-17"	"N3-RL2-18"	"N3-RL2-19"	"N3-RL2-20"	"N3-RL2-21"	"N3-RL2-22"	"N3-RL2-23"	"N3-RL2-24"	"N3-RL2-25"	"N3-RL2-26"	"N3-RL2-27"	"N3-RL2-28"	"N3-RL2-29"	"N3-RL2-30"	"N3-RL2-31"	"N3-RL2-32"	"N3-RL2-33"	"N3-RL2-34"	"N3-RL2-35"	"N3-RL2-36"	"N3-RL2-37"	"N3-RL2-38"	"N3-RL2-39"	"N3-RL2-40"	"N3-RL2-41"	"N3-RL2-42"	"N3-RL2-43"	"N3-RL2-44"	"N3-RL2-45"	"N3-RL2-46"	"N3-RL2-47"	"N3-RL2-48"	"N3-RL2-49"	"N3-RL2-50"	"N3-RL2-51"	"N3-RL2-52"	"N3-RL2-53"	"N3-RL2-54"	"N3-RL2-55"	"N3-RL2-56"	"N3-RL2-57"	"N3-RL2-58"	"N3-RL2-59"	"N3-RL2-60"	"N3-RL2-61"	"N3-RL2-62"	"N3-RL2-63"	"N3-RL2-64"	"N3-RL2-65"	"N3-RL2-66"	"N3-RL2-67"	"N3-RL2-68"	"N3-RL2-69"	"N3-RL2-70"	"N3-RL2-71"	"N3-RL2-72"	"N3-RL2-73"	"N3-RL2-74"	"N3-RL2-75"	"N3-RL2-76"	"N3-RL2-77"	"N3-RL2-78"	"N3-RL2-79"	"N3-RL2-80"	"N3-RL2-82"	"N3-RL2-83"	"N3-RL2-84"	"N3-RL2-85"	"N3-RL2-86"	"N3-RL2-87"	"N3-RL2-88"	"N3-RL2-90"	"N3-RL2-91"	"N3-RL2-92"	"N3-RT11-1"	"N3-RT11-2"	"N3-RT11-3"	"N3-RT11-4"	"N3-RT11-5"	"N3-RT11-6"	"N3-RT11-7"	"N3-RT11-8"	"N3-RT11-9"	"N3-RT11-10"	"N3-RT11-11"	"N3-RT11-12"	"N3-RT11-13"	"N3-RT11-14"	"N3-RT11-15"	"N3-RT11-16"	"N3-RT11-17"	"N3-RT11-18"	"N3-RT11-19"	"N3-RT11-20"	"N3-RT11-21"	"N3-RT11-22"	"N3-RT11-23"	"N3-RT11-24"	"N3-RT11-25"	"N3-RT11-26"	"N3-RT11-27"	"N3-RT11-28"	"N3-RT11-29"	"N3-RT11-30"	"N3-RT11-31"	"N3-RT11-32"	"N3-RT11-33"	"N3-RT11-34"	"N3-RT11-35"	"N3-RT11-36"	"N3-RT11-37"	"N3-RT11-38"	"N3-RT11-39"	"N3-RT11-40"	"N3-RT11-41"	"N3-RT11-42"	"N3-RT11-43"	"N3-RT11-44"	"N3-RT11-45"	"N3-RT11-46"	"N3-RT11-47"	"N3-RT11-48"	"N3-RT11-49"	"N3-RT11-50"	"N3-RT11-51"	"N3-RT11-52"	"N3-RT11-53"	"N3-RT11-54"	"N3-RT11-55"	"N3-RT11-56"	"N3-RT11-57"	"N3-RT11-58"	"N3-RT11-59"	"N3-RT11-60"	"N3-RT11-61"	"N3-RT11-62"	"N3-RT11-63"	"N3-RT11-64"	"N3-RT11-65"	"N3-RT11-66"	"N3-RT11-67"	"N3-RT11-68"	"N3-RT11-69"	"N3-RT11-70"	"N3-RT11-71"	"N3-RT11-72"	"N3-RT11-73"	"N3-RT11-74"	"N3-RT11-75"	"N3-RT11-76"	"N3-RT11-77"	"N3-RT11-78"	"N3-RT11-79"	"N3-RT11-80"	"N3-RT11-81"	"N3-RT11-82"	"N3-RT11-83"	"N3-RT11-84"	"N3-RT11-85"	"N3-RT11-86"	"N3-RT11-87"	"N3-RT11-88"	"N3-RT11-89"	"N3-RT11-90"	"N3-RT11-91"	"N3-RT11-92"	"N3-RT11-93"	"N3-RT11-94"	"N3-RT11-95"	"N3-RT11-96"	"N2-RL5-97"	"N2-RL5-98"	"N2-RL5-99"	"N2-RL5-100"	"N2-RL5-101"	"N2-RL5-102"	"N2-RL5-103"	"N2-RL5-104"	"N2-RL5-105"	"N2-RL5-106"	"N2-RL5-107"	"N2-RL5-108"	"N2-RL5-109"	"N2-RL5-110"	"N2-RL5-111"	"N2-RL5-112"	"N2-RL5-113"	"N2-RL5-114"	"N2-RL5-115"	"N2-RL5-116"	"N2-RL5-117"	"N2-RL5-118"	"N2-RL5-119"	"N2-RL5-120"	"N2-RL5-121"	"N2-RL5-122"	"N2-RL5-123"	"N2-RL5-124"	"N2-RL5-125"	"N2-RL5-126"	"N2-RL5-127"	"N2-RL5-128"	"N2-RT12-97"	"N2-RT12-98"	"N2-RT12-99"	"N2-RT12-100"	"N2-RT12-101"	"N2-RT12-102"	"N2-RT12-103"	"N2-RT12-104"	"N2-RT12-105"	"N2-RT12-106"	"N2-RT12-107"	"N2-RT12-108"	"N2-RT12-109"	"N2-RT12-110"	"N2-RT12-111"	"N2-RT12-112"	"N2-RT12-113"	"N2-RT12-114"	"N2-RT12-115"	"N2-RT12-116"	"N2-RT12-117"	"N2-RT12-118"	"N2-RT12-119"	"N2-RT12-120"	"N2-RT12-121"	"N2-RT12-122"	"N2-RT12-123"	"N2-RT12-124"	"N2-RT12-125"	"N2-RT12-126"	"N2-RT12-127"	"N2-RT12-128"	"N3-RL2-93"	"N3-RL2-94"	"N3-RL2-95"	"N3-RL2-96"	"N3-RL2-97"	"N3-RL2-98"	"N3-RL2-99"	"N3-RL2-100"	"N3-RL2-101"	"N3-RL2-102"	"N3-RL2-103"	"N3-RL2-104"	"N3-RL2-105"	"N3-RL2-106"	"N3-RL2-107"	"N3-RL2-108"	"N3-RL2-109"	"N3-RL2-110"	"N3-RL2-111"	"N3-RL2-112"	"N3-RL2-113"	"N3-RL2-114"	"N3-RL2-115"	"N3-RL2-116"	"N3-RL2-117"	"N3-RL2-118"	"N3-RL2-119"	"N3-RL2-120"	"N3-RL2-121"	"N3-RL2-123"	"N3-RL2-124"	"N3-RT11-97"	"N3-RT11-98"	"N3-RT11-99"	"N3-RT11-100"	"N3-RT11-101"	"N3-RT11-102"	"N3-RT11-103"	"N3-RT11-104"	"N3-RT11-105"	"N3-RT11-106"	"N3-RT11-107"	"N3-RT11-108"	"N3-RT11-109"	"N3-RT11-110"	"N3-RT11-111"	"N3-RT11-112"	"N3-RT11-113"	"N3-RT11-114"	"N3-RT11-115"	"N3-RT11-116"	"N3-RT11-117"	"N3-RT11-118"	"N3-RT11-119"	"N3-RT11-120"	"N3-RT11-121"	"N3-RT11-122"	"N3-RT11-123"	"N3-RT11-124"	"N3-RT11-125"	"N3-RT11-126"	"N3-RT11-127"	"N3-RT11-128"	"N4-RL3-1"	"N4-RL3-2"	"N4-RL3-3"	"N4-RL3-4"	"N4-RL3-5"	"N4-RL3-6"	"N4-RL3-7"	"N4-RL3-8"	"N4-RL3-9"	"N4-RL3-10"	"N4-RL3-11"	"N4-RL3-12"	"N4-RL3-13"	"N4-RL3-14"	"N4-RL3-15"	"N4-RL3-16"	"N4-RL3-17"	"N4-RL3-18"	"N4-RL3-19"	"N4-RL3-20"	"N4-RL3-21"	"N4-RL3-22"	"N4-RL3-23"	"N4-RL3-24"	"N4-RL3-25"	"N4-RL3-26"	"N4-RL3-27"	"N4-RL3-28"	"N4-RL3-29"	"N4-RL3-30"	"N4-RL3-31"	"N4-RL3-32"	"N4-RL3-33"	"N4-RL3-34"	"N4-RL3-35"	"N4-RL3-36"	"N4-RL3-37"	"N4-RL3-38"	"N4-RL3-39"	"N4-RL3-40"	"N4-RL3-41"	"N4-RL3-42"	"N4-RL3-43"	"N4-RL3-44"	"N4-RL3-45"	"N4-RL3-46"	"N4-RL3-47"	"N4-RL3-48"	"N4-RL3-49"	"N4-RL3-50"	"N4-RL3-51"	"N4-RL3-52"	"N4-RL3-53"	"N4-RL3-54"	"N4-RL3-55"	"N4-RL3-56"	"N4-RL3-57"	"N4-RL3-58"	"N4-RL3-59"	"N4-RL3-60"	"N4-RL3-61"	"N4-RL3-62"	"N4-RL3-63"	"N4-RL3-64"	"N4-RL3-65"	"N4-RL3-66"	"N4-RL3-67"	"N4-RL3-68"	"N4-RL3-69"	"N4-RL3-70"	"N4-RL3-71"	"N4-RL3-72"	"N4-RL3-73"	"N4-RL3-74"	"N4-RL3-75"	"N4-RL3-76"	"N4-RL3-77"	"N4-RL3-78"	"N4-RL3-79"	"N4-RL3-80"	"N4-RL3-82"	"N4-RL3-83"	"N4-RL3-84"	"N4-RL3-85"	"N4-RL3-86"	"N4-RL3-87"	"N4-RL3-88"	"N4-RL3-89"	"N4-RL3-90"	"N4-RL3-91"	"N4-RL3-92"	"N4-RL3-93"	"N4-RL3-94"	"N4-RL3-95"	"N4-RL3-96"	"N4-RL3-97"	"N4-RL3-98"	"N4-RL3-99"	"N4-RL3-100"	"N4-RL3-101"	"N4-RL3-102"	"N4-RL3-103"	"N4-RL3-104"	"N4-RL3-105"	"N4-RL3-106"	"N4-RL3-107"	"N4-RL3-108"	"N4-RL3-109"	"N4-RL3-110"	"N4-RL3-111"	"N4-RL3-112"	"N4-RL3-113"	"N4-RL3-114"	"N4-RL3-115"	"N4-RL3-116"	"N4-RL3-117"	"N4-RL3-119"	"N4-RL3-120"	"N4-RL3-121"	"N4-RL3-122"	"N4-RL3-124"	"N4-RL3-125"	"N4-RL3-126"	"N4-RL3-127"	"N4-RL3-128"	"N4-RL3-129"	"N4-RL3-131"	"N4-RL3-132"	"N4-RT12-1"	"N4-RT12-2"	"N4-RT12-3"	"N4-RT12-4"	"N4-RT12-5"	"N4-RT12-6"	"N4-RT12-7"	"N4-RT12-8"	"N4-RT12-9"	"N4-RT12-10"	"N4-RT12-11"	"N4-RT12-12"	"N4-RT12-13"	"N4-RT12-14"	"N4-RT12-15"	"N4-RT12-16"	"N4-RT12-17"	"N4-RT12-18"	"N4-RT12-19"	"N4-RT12-20"	"N4-RT12-21"	"N4-RT12-22"	"N4-RT12-23"	"N4-RT12-24"	"N4-RT12-25"	"N4-RT12-26"	"N4-RT12-27"	"N4-RT12-28"	"N4-RT12-29"	"N4-RT12-30"	"N4-RT12-31"	"N4-RT12-32"	"N4-RT12-33"	"N4-RT12-34"	"N4-RT12-35"	"N4-RT12-36"	"N4-RT12-37"	"N4-RT12-38"	"N4-RT12-39"	"N4-RT12-40"	"N4-RT12-41"	"N4-RT12-42"	"N4-RT12-43"	"N4-RT12-44"	"N4-RT12-45"	"N4-RT12-46"	"N4-RT12-47"	"N4-RT12-48"	"N4-RT12-49"	"N4-RT12-50"	"N4-RT12-51"	"N4-RT12-52"	"N4-RT12-53"	"N4-RT12-54"	"N4-RT12-55"	"N4-RT12-56"	"N4-RT12-57"	"N4-RT12-58"	"N4-RT12-59"	"N4-RT12-60"	"N4-RT12-61"	"N4-RT12-62"	"N4-RT12-63"	"N4-RT12-64"	"N4-RT12-65"	"N4-RT12-66"	"N4-RT12-67"	"N4-RT12-68"	"N4-RT12-69"	"N4-RT12-70"	"N4-RT12-71"	"N4-RT12-72"	"N4-RT12-73"	"N4-RT12-74"	"N4-RT12-75"	"N4-RT12-76"	"N4-RT12-77"	"N4-RT12-78"	"N4-RT12-79"	"N4-RT12-80"	"N4-RT12-81"	"N4-RT12-82"	"N4-RT12-83"	"N4-RT12-84"	"N4-RT12-85"	"N4-RT12-86"	"N4-RT12-87"	"N4-RT12-88"	"N4-RT12-89"	"N4-RT12-90"	"N4-RT12-91"	"N4-RT12-92"	"N4-RT12-93"	"N4-RT12-94"	"N4-RT12-95"	"N4-RT12-96"	"N4-RT12-97"	"N4-RT12-98"	"N4-RT12-99"	"N4-RT12-100"	"N4-RT12-101"	"N4-RT12-102"	"N4-RT12-103"	"N4-RT12-104"	"N4-RT12-105"	"N4-RT12-106"	"N4-RT12-107"	"N4-RT12-108"	"N4-RT12-109"	"N4-RT12-110"	"N4-RT12-111"	"N4-RT12-112"	"N4-RT12-113"	"N4-RT12-114"	"N4-RT12-115"	"N4-RT12-116"	"N4-RT12-117"	"N4-RT12-118"	"N4-RT12-119"	"N4-RT12-120"	"N4-RT12-122"	"N4-RT12-123"	"N4-RT12-124"	"N4-RT12-125"	"N4-RT12-126"	"N4-RT12-127"	"N4-RT12-128"	"N4-RT12-129"	"N2-RL5-129"	"N2-RL5-130"	"N2-RL5-131"	"N2-RL5-132"	"N2-RL5-133"	"N2-RL5-134"	"N2-RL5-135"	"N2-RL5-136"	"N2-RL5-137"	"N2-RL5-138"	"N2-RL5-139"	"N2-RL5-140"	"N2-RL5-141"	"N2-RL5-142"	"N2-RL5-143"	"N2-RL5-144"	"N2-RL5-145"	"N2-RL5-146"	"N2-RL5-147"	"N2-RL5-148"	"N2-RL5-149"	"N2-RL5-150"	"N2-RL5-151"	"N2-RL5-152"	"N2-RL5-153"	"N2-RL5-154"	"N2-RL5-155"	"N2-RL5-156"	"N2-RL5-157"	"N2-RL5-158"	"N2-RL5-159"	"N2-RL5-160"	"N2-RL5-162"	"N2-RL5-163"	"N2-RL5-164"	"N2-RL5-165"	"N2-RL5-166"	"N2-RL5-167"	"N2-RL5-169"	"N2-RL5-170"	"N2-RL5-171"	"N2-RL5-173"	"N2-RL5-174"	"N2-RL5-175"	"N2-RL5-176"	"N2-RL5-177"	"N2-RL5-178"	"N2-RL5-179"	"N2-RL5-180"	"N2-RL5-181"	"N2-RL5-182"	"N2-RL5-183"	"N2-RL5-184"	"N2-RL5-185"	"N2-RL5-186"	"N2-RL5-187"	"N2-RL5-188"	"N2-RL5-189"	"N2-RL5-190"	"N2-RL5-191"	"N2-RL5-192"	"N2-RL5-193"	"N2-RT12-129"	"N2-RT12-130"	"N2-RT12-131"	"N2-RT12-132"	"N2-RT12-133"	"N2-RT12-134"	"N2-RT12-135"	"N2-RT12-136"	"N2-RT12-137"	"N2-RT12-138"	"N2-RT12-139"	"N2-RT12-140"	"N2-RT12-141"	"N2-RT12-142"	"N2-RT12-143"	"N2-RT12-144"	"N2-RT12-145"	"N2-RT12-146"	"N2-RT12-147"	"N2-RT12-148"	"N2-RT12-149"	"N2-RT12-150"	"N2-RT12-151"	"N2-RT12-152"	"N2-RT12-153"	"N2-RT12-154"	"N2-RT12-155"	"N2-RT12-156"	"N2-RT12-157"	"N2-RT12-158"	"N2-RT12-159"	"N2-RT12-160"	"N2-RT12-161"	"N2-RT12-162"	"N2-RT12-163"	"N2-RT12-164"	"N2-RT12-165"	"N2-RT12-166"	"N2-RT12-167"	"N2-RT12-168"	"N2-RT12-169"	"N2-RT12-170"	"N2-RT12-171"	"N2-RT12-172"	"N2-RT12-173"	"N2-RT12-174"	"N2-RT12-175"	"N2-RT12-176"	"N2-RT12-177"	"N2-RT12-178"	"N2-RT12-179"	"N2-RT12-180"	"N2-RT12-181"	"N2-RT12-182"	"N2-RT12-183"	"N2-RT12-184"	"N2-RT12-185"	"N2-RT12-186"	"N2-RT12-187"	"N2-RT12-188"	"N2-RT12-189"	"N2-RT12-190"	"N2-RT12-191"	"N2-RT12-192"	"N3-RL2-125"	"N3-RL2-126"	"N3-RL2-127"	"N3-RL2-128"	"N3-RL2-129"	"N3-RL2-130"	"N3-RL2-131"	"N3-RL2-132"	"N3-RL2-133"	"N3-RL2-134"	"N3-RL2-135"	"N3-RL2-136"	"N3-RL2-137"	"N3-RL2-138"	"N3-RL2-139"	"N3-RL2-140"	"N3-RL2-141"	"N3-RL2-142"	"N3-RL2-143"	"N3-RL2-144"	"N3-RL2-145"	"N3-RL2-146"	"N3-RL2-147"	"N3-RL2-148"	"N3-RL2-149"	"N3-RL2-150"	"N3-RL2-151"	"N3-RL2-152"	"N3-RL2-153"	"N3-RL2-154"	"N3-RL2-155"	"N3-RL2-156"	"N3-RL2-157"	"N3-RL2-158"	"N3-RL2-159"	"N3-RL2-160"	"N3-RL2-161"	"N3-RL2-162"	"N3-RL2-163"	"N3-RL2-164"	"N3-RL2-165"	"N3-RL2-166"	"N3-RL2-167"	"N3-RL2-168"	"N3-RL2-169"	"N3-RL2-170"	"N3-RL2-171"	"N3-RL2-172"	"N3-RL2-173"	"N3-RL2-174"	"N3-RL2-175"	"N3-RL2-176"	"N3-RL2-177"	"N3-RL2-178"	"N3-RL2-179"	"N3-RL2-180"	"N3-RL2-181"	"N3-RL2-182"	"N3-RL2-183"	"N3-RL2-184"	"N3-RL2-185"	"N3-RL2-186"	"N3-RL2-187"	"N3-RL2-188"	"N3-RT11-129"	"N3-RT11-130"	"N3-RT11-131"	"N3-RT11-132"	"N3-RT11-133"	"N3-RT11-134"	"N3-RT11-135"	"N3-RT11-136"	"N3-RT11-137"	"N3-RT11-138"	"N3-RT11-139"	"N3-RT11-140"	"N3-RT11-141"	"N3-RT11-142"	"N3-RT11-143"	"N3-RT11-144"	"N3-RT11-145"	"N3-RT11-146"	"N3-RT11-147"	"N3-RT11-148"	"N3-RT11-149"	"N3-RT11-150"	"N3-RT11-151"	"N3-RT11-152"	"N3-RT11-153"	"N3-RT11-154"	"N3-RT11-155"	"N3-RT11-156"	"N3-RT11-157"	"N3-RT11-158"	"N3-RT11-159"	"N3-RT11-160"	"N3-RT11-161"	"N3-RT11-162"	"N3-RT11-163"	"N3-RT11-164"	"N3-RT11-165"	"N3-RT11-166"	"N3-RT11-167"	"N3-RT11-168"	"N3-RT11-169"	"N3-RT11-170"	"N3-RT11-171"	"N3-RT11-172"	"N3-RT11-173"	"N3-RT11-174"	"N3-RT11-175"	"N3-RT11-176"	"N3-RT11-177"	"N3-RT11-178"	"N3-RT11-179"	"N3-RT11-180"	"N3-RT11-181"	"N3-RT11-182"	"N3-RT11-183"	"N3-RT11-184"	"N3-RT11-185"	"N3-RT11-186"	"N3-RT11-187"	"N3-RT11-188"	"N3-RT11-189"	"N3-RT11-190"	"N3-RT11-191"	"N3-RT11-192"	"N4-RL3-133"	"N4-RL3-134"	"N4-RL3-135"	"N4-RL3-136"	"N4-RL3-137"	"N4-RL3-138"	"N4-RL3-139"	"N4-RL3-140"	"N4-RL3-141"	"N4-RL3-142"	"N4-RL3-143"	"N4-RL3-144"	"N4-RL3-145"	"N4-RL3-146"	"N4-RL3-147"	"N4-RL3-148"	"N4-RL3-149"	"N4-RL3-150"	"N4-RL3-151"	"N4-RL3-152"	"N4-RL3-153"	"N4-RL3-154"	"N4-RL3-155"	"N4-RL3-156"	"N4-RL3-157"	"N4-RL3-158"	"N4-RL3-159"	"N4-RL3-161"	"N4-RL3-162"	"N4-RL3-163"	"N4-RL3-164"	"N4-RL3-165"	"N4-RL3-166"	"N4-RL3-167"	"N4-RL3-168"	"N4-RL3-169"	"N4-RL3-170"	"N4-RL3-171"	"N4-RL3-172"	"N4-RL3-173"	"N4-RL3-174"	"N4-RL3-175"	"N4-RL3-176"	"N4-RL3-177"	"N4-RL3-180"	"N4-RL3-181"	"N4-RL3-182"	"N4-RL3-183"	"N4-RL3-184"	"N4-RL3-185"	"N4-RL3-186"	"N4-RL3-187"	"N4-RL3-188"	"N4-RL3-189"	"N4-RL3-190"	"N4-RL3-191"	"N4-RL3-192"	"N4-RL3-193"	"N4-RL3-194"	"N4-RL3-195"	"N4-RL3-196"	"N4-RL3-197"	"N4-RL3-198"	"N4-RL3-199"	"N4-RT12-130"	"N4-RT12-131"	"N4-RT12-132"	"N4-RT12-133"	"N4-RT12-134"	"N4-RT12-135"	"N4-RT12-136"	"N4-RT12-137"	"N4-RT12-138"	"N4-RT12-139"	"N4-RT12-140"	"N4-RT12-141"	"N4-RT12-142"	"N4-RT12-143"	"N4-RT12-144"	"N4-RT12-145"	"N4-RT12-146"	"N4-RT12-147"	"N4-RT12-148"	"N4-RT12-149"	"N4-RT12-150"	"N4-RT12-151"	"N4-RT12-152"	"N4-RT12-154"	"N4-RT12-155"	"N4-RT12-156"	"N4-RT12-157"	"N4-RT12-158"	"N4-RT12-159"	"N4-RT12-160"	"N4-RT12-161"	"N4-RT12-162"	"N4-RT12-163"	"N4-RT12-164"	"N4-RT12-165"	"N4-RT12-166"	"N4-RT12-167"	"N4-RT12-168"	"N4-RT12-169"	"N4-RT12-170"	"N4-RT12-171"	"N4-RT12-172"	"N4-RT12-173"	"N4-RT12-174"	"N4-RT12-175"	"N4-RT12-176"	"N4-RT12-177"	"N4-RT12-178"	"N4-RT12-179"	"N4-RT12-180"	"N4-RT12-181"	"N4-RT12-182"	"N4-RT12-183"	"N4-RT12-184"	"N4-RT12-185"	"N4-RT12-186"	"N4-RT12-187"	"N4-RT12-188"	"N4-RT12-189"	"N4-RT12-190"	"N4-RT12-191"	"N4-RT12-192"	"N4-RT12-193"	"N4-RT12-194"
gsm	"GSM7962467"	"GSM7962468"	"GSM7962469"	"GSM7962470"	"GSM7962471"	"GSM7962472"	"GSM7962473"	"GSM7962474"	"GSM7962475"	"GSM7962476"	"GSM7962477"	"GSM7962478"	"GSM7962479"	"GSM7962480"	"GSM7962481"	"GSM7962482"	"GSM7962483"	"GSM7962484"	"GSM7962485"	"GSM7962486"	"GSM7962487"	"GSM7962488"	"GSM7962489"	"GSM7962490"	"GSM7962491"	"GSM7962492"	"GSM7962493"	"GSM7962494"	"GSM7962495"	"GSM7962496"	"GSM7962497"	"GSM7962498"	"GSM7962499"	"GSM7962500"	"GSM7962501"	"GSM7962502"	"GSM7962503"	"GSM7962504"	"GSM7962505"	"GSM7962506"	"GSM7962507"	"GSM7962508"	"GSM7962509"	"GSM7962510"	"GSM7962511"	"GSM7962512"	"GSM7962513"	"GSM7962514"	"GSM7962515"	"GSM7962516"	"GSM7962517"	"GSM7962518"	"GSM7962519"	"GSM7962520"	"GSM7962521"	"GSM7962522"	"GSM7962523"	"GSM7962524"	"GSM7962525"	"GSM7962526"	"GSM7962527"	"GSM7962528"	"GSM7962529"	"GSM7962530"	"GSM7962531"	"GSM7962532"	"GSM7962533"	"GSM7962534"	"GSM7962535"	"GSM7962536"	"GSM7962537"	"GSM7962538"	"GSM7962539"	"GSM7962540"	"GSM7962541"	"GSM7962542"	"GSM7962543"	"GSM7962544"	"GSM7962545"	"GSM7962546"	"GSM7962547"	"GSM7962548"	"GSM7962549"	"GSM7962550"	"GSM7962551"	"GSM7962552"	"GSM7962553"	"GSM7962554"	"GSM7962555"	"GSM7962556"	"GSM7962557"	"GSM7962558"	"GSM7962559"	"GSM7962560"	"GSM7962561"	"GSM7962562"	"GSM7962563"	"GSM7962564"	"GSM7962565"	"GSM7962566"	"GSM7962567"	"GSM7962568"	"GSM7962569"	"GSM7962570"	"GSM7962571"	"GSM7962572"	"GSM7962573"	"GSM7962574"	"GSM7962575"	"GSM7962576"	"GSM7962577"	"GSM7962578"	"GSM7962579"	"GSM7962580"	"GSM7962581"	"GSM7962582"	"GSM7962583"	"GSM7962584"	"GSM7962585"	"GSM7962586"	"GSM7962587"	"GSM7962588"	"GSM7962589"	"GSM7962590"	"GSM7962591"	"GSM7962592"	"GSM7962593"	"GSM7962594"	"GSM7962595"	"GSM7962596"	"GSM7962597"	"GSM7962598"	"GSM7962599"	"GSM7962600"	"GSM7962601"	"GSM7962602"	"GSM7962603"	"GSM7962604"	"GSM7962605"	"GSM7962606"	"GSM7962607"	"GSM7962608"	"GSM7962609"	"GSM7962610"	"GSM7962611"	"GSM7962612"	"GSM7962613"	"GSM7962614"	"GSM7962615"	"GSM7962616"	"GSM7962617"	"GSM7962618"	"GSM7962619"	"GSM7962620"	"GSM7962621"	"GSM7962622"	"GSM7962623"	"GSM7962624"	"GSM7962625"	"GSM7962626"	"GSM7962627"	"GSM7962628"	"GSM7962629"	"GSM7962630"	"GSM7962631"	"GSM7962632"	"GSM7962633"	"GSM7962634"	"GSM7962635"	"GSM7962636"	"GSM7962637"	"GSM7962638"	"GSM7962639"	"GSM7962640"	"GSM7962641"	"GSM7962642"	"GSM7962643"	"GSM7962644"	"GSM7962645"	"GSM7962646"	"GSM7962647"	"GSM7962648"	"GSM7962649"	"GSM7962650"	"GSM7962651"	"GSM7962652"	"GSM7962653"	"GSM7962654"	"GSM7962655"	"GSM7962656"	"GSM7962657"	"GSM7962658"	"GSM7962659"	"GSM7962660"	"GSM7962661"	"GSM7962662"	"GSM7962663"	"GSM7962664"	"GSM7962665"	"GSM7962666"	"GSM7962667"	"GSM7962668"	"GSM7962669"	"GSM7962670"	"GSM7962671"	"GSM7962672"	"GSM7962673"	"GSM7962674"	"GSM7962675"	"GSM7962676"	"GSM7962677"	"GSM7962678"	"GSM7962679"	"GSM7962680"	"GSM7962681"	"GSM7962682"	"GSM7962683"	"GSM7962684"	"GSM7962685"	"GSM7962686"	"GSM7962687"	"GSM7962688"	"GSM7962689"	"GSM7962690"	"GSM7962691"	"GSM7962692"	"GSM7962693"	"GSM7962694"	"GSM7962695"	"GSM7962696"	"GSM7962697"	"GSM7962698"	"GSM7962699"	"GSM7962700"	"GSM7962701"	"GSM7962702"	"GSM7962703"	"GSM7962704"	"GSM7962705"	"GSM7962706"	"GSM7962707"	"GSM7962708"	"GSM7962709"	"GSM7962710"	"GSM7962711"	"GSM7962712"	"GSM7962713"	"GSM7962714"	"GSM7962715"	"GSM7962716"	"GSM7962717"	"GSM7962718"	"GSM7962719"	"GSM7962720"	"GSM7962721"	"GSM7962722"	"GSM7962723"	"GSM7962724"	"GSM7962725"	"GSM7962726"	"GSM7962727"	"GSM7962728"	"GSM7962729"	"GSM7962730"	"GSM7962731"	"GSM7962732"	"GSM7962733"	"GSM7962734"	"GSM7962735"	"GSM7962736"	"GSM7962737"	"GSM7962738"	"GSM7962739"	"GSM7962740"	"GSM7962741"	"GSM7962742"	"GSM7962743"	"GSM7962744"	"GSM7962745"	"GSM7962746"	"GSM7962747"	"GSM7962748"	"GSM7962749"	"GSM7962750"	"GSM7962751"	"GSM7962752"	"GSM7962753"	"GSM7962754"	"GSM7962755"	"GSM7962756"	"GSM7962757"	"GSM7962758"	"GSM7962759"	"GSM7962760"	"GSM7962761"	"GSM7962762"	"GSM7962763"	"GSM7962764"	"GSM7962765"	"GSM7962766"	"GSM7962767"	"GSM7962768"	"GSM7962769"	"GSM7962770"	"GSM7962771"	"GSM7962772"	"GSM7962773"	"GSM7962774"	"GSM7962775"	"GSM7962776"	"GSM7962777"	"GSM7962778"	"GSM7962779"	"GSM7962780"	"GSM7962781"	"GSM7962782"	"GSM7962783"	"GSM7962784"	"GSM7962785"	"GSM7962786"	"GSM7962787"	"GSM7962788"	"GSM7962789"	"GSM7962790"	"GSM7962791"	"GSM7962792"	"GSM7962793"	"GSM7962794"	"GSM7962795"	"GSM7962796"	"GSM7962797"	"GSM7962798"	"GSM7962799"	"GSM7962800"	"GSM7962801"	"GSM7962802"	"GSM7962803"	"GSM7962804"	"GSM7962805"	"GSM7962806"	"GSM7962807"	"GSM7962808"	"GSM7962809"	"GSM7962810"	"GSM7962811"	"GSM7962812"	"GSM7962813"	"GSM7962814"	"GSM7962815"	"GSM7962816"	"GSM7962817"	"GSM7962818"	"GSM7962819"	"GSM7962820"	"GSM7962821"	"GSM7962822"	"GSM7962823"	"GSM7962824"	"GSM7962825"	"GSM7962826"	"GSM7962827"	"GSM7962828"	"GSM7962829"	"GSM7962830"	"GSM7962831"	"GSM7962832"	"GSM7962833"	"GSM7962834"	"GSM7962835"	"GSM7962836"	"GSM7962837"	"GSM7962838"	"GSM7962839"	"GSM7962840"	"GSM7962841"	"GSM7962842"	"GSM7962843"	"GSM7962844"	"GSM7962845"	"GSM7962846"	"GSM7962847"	"GSM7962848"	"GSM7962849"	"GSM7962850"	"GSM7962851"	"GSM7962852"	"GSM7962853"	"GSM7962854"	"GSM7962855"	"GSM7962856"	"GSM7962857"	"GSM7962858"	"GSM7962859"	"GSM7962860"	"GSM7962861"	"GSM7962862"	"GSM7962863"	"GSM7962864"	"GSM7962865"	"GSM7962866"	"GSM7962867"	"GSM7962868"	"GSM7962869"	"GSM7962870"	"GSM7962871"	"GSM7962872"	"GSM7962873"	"GSM7962874"	"GSM7962875"	"GSM7962876"	"GSM7962877"	"GSM7962878"	"GSM7962879"	"GSM7962880"	"GSM7962881"	"GSM7962882"	"GSM7962883"	"GSM7962884"	"GSM7962885"	"GSM7962886"	"GSM7962887"	"GSM7962888"	"GSM7962889"	"GSM7962890"	"GSM7962891"	"GSM7962892"	"GSM7962893"	"GSM7962894"	"GSM7962895"	"GSM7962896"	"GSM7962897"	"GSM7962898"	"GSM7962899"	"GSM7962900"	"GSM7962901"	"GSM7962902"	"GSM7962903"	"GSM7962904"	"GSM7962905"	"GSM7962906"	"GSM7962907"	"GSM7962908"	"GSM7962909"	"GSM7962910"	"GSM7962911"	"GSM7962912"	"GSM7962913"	"GSM7962914"	"GSM7962915"	"GSM7962916"	"GSM7962917"	"GSM7962918"	"GSM7962919"	"GSM7962920"	"GSM7962921"	"GSM7962922"	"GSM7962923"	"GSM7962924"	"GSM7962925"	"GSM7962926"	"GSM7962927"	"GSM7962928"	"GSM7962929"	"GSM7962930"	"GSM7962931"	"GSM7962932"	"GSM7962933"	"GSM7962934"	"GSM7962935"	"GSM7962936"	"GSM7962937"	"GSM7962938"	"GSM7962939"	"GSM7962940"	"GSM7962941"	"GSM7962942"	"GSM7962943"	"GSM7962944"	"GSM7962945"	"GSM7962946"	"GSM7962947"	"GSM7962948"	"GSM7962949"	"GSM7962950"	"GSM7962951"	"GSM7962952"	"GSM7962953"	"GSM7962954"	"GSM7962955"	"GSM7962956"	"GSM7962957"	"GSM7962958"	"GSM7962959"	"GSM7962960"	"GSM7962961"	"GSM7962962"	"GSM7962963"	"GSM7962964"	"GSM7962965"	"GSM7962966"	"GSM7962967"	"GSM7962968"	"GSM7962969"	"GSM7962970"	"GSM7962971"	"GSM7962972"	"GSM7962973"	"GSM7962974"	"GSM7962975"	"GSM7962976"	"GSM7962977"	"GSM7962978"	"GSM7962979"	"GSM7962980"	"GSM7962981"	"GSM7962982"	"GSM7962983"	"GSM7962984"	"GSM7962985"	"GSM7962986"	"GSM7962987"	"GSM7962988"	"GSM7962989"	"GSM7962990"	"GSM7962991"	"GSM7962992"	"GSM7962993"	"GSM7962994"	"GSM7962995"	"GSM7962996"	"GSM7962997"	"GSM7962998"	"GSM7962999"	"GSM7963000"	"GSM7963001"	"GSM7963002"	"GSM7963003"	"GSM7963004"	"GSM7963005"	"GSM7963006"	"GSM7963007"	"GSM7963008"	"GSM7963009"	"GSM7963010"	"GSM7963011"	"GSM7963012"	"GSM7963013"	"GSM7963014"	"GSM7963015"	"GSM7963016"	"GSM7963017"	"GSM7963018"	"GSM7963019"	"GSM7963020"	"GSM7963021"	"GSM7963022"	"GSM7963023"	"GSM7963024"	"GSM7963025"	"GSM7963026"	"GSM7963027"	"GSM7963028"	"GSM7963029"	"GSM7963030"	"GSM7963031"	"GSM7963032"	"GSM7963033"	"GSM7963034"	"GSM7963035"	"GSM7963036"	"GSM7963037"	"GSM7963038"	"GSM7963039"	"GSM7963040"	"GSM7963041"	"GSM7963042"	"GSM7963043"	"GSM7963044"	"GSM7963045"	"GSM7963046"	"GSM7963047"	"GSM7963048"	"GSM7963049"	"GSM7963050"	"GSM7963051"	"GSM7963052"	"GSM7963053"	"GSM7963054"	"GSM7963055"	"GSM7963056"	"GSM7963057"	"GSM7963058"	"GSM7963059"	"GSM7963060"	"GSM7963061"	"GSM7963062"	"GSM7963063"	"GSM7963064"	"GSM7963065"	"GSM7963066"	"GSM7963067"	"GSM7963068"	"GSM7963069"	"GSM7963070"	"GSM7963071"	"GSM7963072"	"GSM7963073"	"GSM7963074"	"GSM7963075"	"GSM7963076"	"GSM7963077"	"GSM7963078"	"GSM7963079"	"GSM7963080"	"GSM7963081"	"GSM7963082"	"GSM7963083"	"GSM7963084"	"GSM7963085"	"GSM7963086"	"GSM7963087"	"GSM7963088"	"GSM7963089"	"GSM7963090"	"GSM7963091"	"GSM7963092"	"GSM7963093"	"GSM7963094"	"GSM7963095"	"GSM7963096"	"GSM7963097"	"GSM7963098"	"GSM7963099"	"GSM7963100"	"GSM7963101"	"GSM7963102"	"GSM7963103"	"GSM7963104"	"GSM7963105"	"GSM7963106"	"GSM7963107"	"GSM7963108"	"GSM7963109"	"GSM7963110"	"GSM7963111"	"GSM7963112"	"GSM7963113"	"GSM7963114"	"GSM7963115"	"GSM7963116"	"GSM7963117"	"GSM7963118"	"GSM7963119"	"GSM7963120"	"GSM7963121"	"GSM7963122"	"GSM7963123"	"GSM7963124"	"GSM7963125"	"GSM7963126"	"GSM7963127"	"GSM7963128"	"GSM7963129"	"GSM7963130"	"GSM7963131"	"GSM7963132"	"GSM7963133"	"GSM7963134"	"GSM7963135"	"GSM7963136"	"GSM7963137"	"GSM7963138"	"GSM7963139"	"GSM7963140"	"GSM7963141"	"GSM7963142"	"GSM7963143"	"GSM7963144"	"GSM7963145"	"GSM7963146"	"GSM7963147"	"GSM7963148"	"GSM7963149"	"GSM7963150"	"GSM7963151"	"GSM7963152"	"GSM7963153"	"GSM7963154"	"GSM7963155"	"GSM7963156"	"GSM7963157"	"GSM7963158"	"GSM7963159"	"GSM7963160"	"GSM7963161"	"GSM7963162"	"GSM7963163"	"GSM7963164"	"GSM7963165"	"GSM7963166"	"GSM7963167"	"GSM7963168"	"GSM7963169"	"GSM7963170"	"GSM7963171"	"GSM7963172"	"GSM7963173"	"GSM7963174"	"GSM7963175"	"GSM7963176"	"GSM7963177"	"GSM7963178"	"GSM7963179"	"GSM7963180"	"GSM7963181"	"GSM7963182"	"GSM7963183"	"GSM7963184"	"GSM7963185"	"GSM7963186"	"GSM7963187"	"GSM7963188"	"GSM7963189"	"GSM7963190"	"GSM7963191"	"GSM7963192"	"GSM7963193"	"GSM7963194"	"GSM7963195"	"GSM7963196"	"GSM7963197"	"GSM7963198"	"GSM7963199"	"GSM7963200"	"GSM7963201"	"GSM7963202"	"GSM7963203"	"GSM7963204"	"GSM7963205"	"GSM7963206"	"GSM7963207"	"GSM7963208"	"GSM7963209"	"GSM7963210"	"GSM7963211"	"GSM7963212"	"GSM7963213"	"GSM7963214"	"GSM7963215"	"GSM7963216"	"GSM7963217"	"GSM7963218"	"GSM7963219"	"GSM7963220"	"GSM7963221"	"GSM7963222"	"GSM7963223"	"GSM7963224"	"GSM7963225"	"GSM7963226"	"GSM7963227"	"GSM7963228"	"GSM7963229"	"GSM7963230"	"GSM7963231"	"GSM7963232"	"GSM7963233"	"GSM7963234"	"GSM7963235"	"GSM7963236"	"GSM7963237"	"GSM7963238"	"GSM7963239"	"GSM7963240"	"GSM7963241"	"GSM7963242"	"GSM7963243"	"GSM7963244"	"GSM7963245"	"GSM7963246"	"GSM7963247"	"GSM7963248"	"GSM7963249"	"GSM7963250"	"GSM7963251"	"GSM7963252"	"GSM7963253"	"GSM7963254"	"GSM7963255"	"GSM7963256"	"GSM7963257"	"GSM7963258"	"GSM7963259"	"GSM7963260"	"GSM7963261"	"GSM7963262"	"GSM7963263"	"GSM7963264"	"GSM7963265"	"GSM7963266"	"GSM7963267"	"GSM7963268"	"GSM7963269"	"GSM7963270"	"GSM7963271"	"GSM7963272"	"GSM7963273"	"GSM7963274"	"GSM7963275"	"GSM7963276"	"GSM7963277"	"GSM7963278"	"GSM7963279"	"GSM7963280"	"GSM7963281"	"GSM7963282"	"GSM7963283"	"GSM7963284"	"GSM7963285"	"GSM7963286"	"GSM7963287"	"GSM7963288"	"GSM7963289"	"GSM7963290"	"GSM7963291"	"GSM7963292"	"GSM7963293"	"GSM7963294"	"GSM7963295"	"GSM7963296"	"GSM7963297"	"GSM7963298"	"GSM7963299"	"GSM7963300"	"GSM7963301"	"GSM7963302"	"GSM7963303"	"GSM7963304"	"GSM7963305"	"GSM7963306"	"GSM7963307"	"GSM7963308"	"GSM7963309"	"GSM7963310"	"GSM7963311"	"GSM7963312"	"GSM7963313"	"GSM7963314"	"GSM7963315"	"GSM7963316"	"GSM7963317"	"GSM7963318"	"GSM7963319"	"GSM7963320"	"GSM7963321"	"GSM7963322"	"GSM7963323"	"GSM7963324"	"GSM7963325"	"GSM7963326"	"GSM7963327"	"GSM7963328"	"GSM7963329"	"GSM7963330"	"GSM7963331"	"GSM7963332"	"GSM7963333"	"GSM7963334"	"GSM7963335"	"GSM7963336"	"GSM7963337"	"GSM7963338"	"GSM7963339"	"GSM7963340"	"GSM7963341"	"GSM7963342"	"GSM7963343"	"GSM7963344"	"GSM7963345"	"GSM7963346"	"GSM7963347"	"GSM7963348"	"GSM7963349"	"GSM7963350"	"GSM7963351"	"GSM7963352"	"GSM7963353"	"GSM7963354"	"GSM7963355"	"GSM7963356"	"GSM7963357"	"GSM7963358"	"GSM7963359"	"GSM7963360"	"GSM7963361"	"GSM7963362"	"GSM7963363"	"GSM7963364"	"GSM7963365"	"GSM7963366"	"GSM7963367"	"GSM7963368"	"GSM7963369"	"GSM7963370"	"GSM7963371"	"GSM7963372"	"GSM7963373"	"GSM7963374"	"GSM7963375"	"GSM7963376"	"GSM7963377"	"GSM7963378"	"GSM7963379"	"GSM7963380"	"GSM7963381"	"GSM7963382"	"GSM7963383"	"GSM7963384"	"GSM7963385"	"GSM7963386"	"GSM7963387"	"GSM7963388"	"GSM7963389"	"GSM7963390"	"GSM7963391"	"GSM7963392"	"GSM7963393"	"GSM7963394"	"GSM7963395"	"GSM7963396"	"GSM7963397"	"GSM7963398"	"GSM7963399"	"GSM7963400"	"GSM7963401"	"GSM7963402"	"GSM7963403"	"GSM7963404"	"GSM7963405"	"GSM7963406"	"GSM7963407"	"GSM7963408"	"GSM7963409"	"GSM7963410"	"GSM7963411"	"GSM7963412"	"GSM7963413"	"GSM7963414"	"GSM7963415"	"GSM7963416"	"GSM7963417"	"GSM7963418"	"GSM7963419"	"GSM7963420"	"GSM7963421"	"GSM7963422"	"GSM7963423"	"GSM7963424"	"GSM7963425"	"GSM7963426"	"GSM7963427"	"GSM7963428"	"GSM7963429"	"GSM7963430"	"GSM7963431"	"GSM7963432"	"GSM7963433"	"GSM7963434"	"GSM7963435"	"GSM7963436"	"GSM7963437"	"GSM7963438"	"GSM7963439"	"GSM7963440"	"GSM7963441"	"GSM7963442"	"GSM7963443"	"GSM7963444"	"GSM7963445"	"GSM7963446"	"GSM7963447"	"GSM7963448"	"GSM7963449"	"GSM7963450"	"GSM7963451"	"GSM7963452"	"GSM7963453"	"GSM7963454"	"GSM7963455"	"GSM7963456"	"GSM7963457"	"GSM7963458"	"GSM7963459"	"GSM7963460"	"GSM7963461"	"GSM7963462"	"GSM7963463"	"GSM7963464"	"GSM7963465"	"GSM7963466"	"GSM7963467"	"GSM7963468"	"GSM7963469"	"GSM7963470"	"GSM7963471"	"GSM7963472"	"GSM7963473"	"GSM7963474"	"GSM7963475"	"GSM7963476"	"GSM7963477"	"GSM7963478"	"GSM7963479"	"GSM7963480"	"GSM7963481"	"GSM7963482"	"GSM7963483"	"GSM7963484"	"GSM7963485"	"GSM7963486"	"GSM7963487"	"GSM7963488"	"GSM7963489"	"GSM7963490"	"GSM7963491"	"GSM7963492"	"GSM7963493"	"GSM7963494"	"GSM7963495"	"GSM7963496"	"GSM7963497"	"GSM7963498"	"GSM7963499"	"GSM7963500"	"GSM7963501"	"GSM7963502"	"GSM7963503"	"GSM7963504"	"GSM7963505"	"GSM7963506"	"GSM7963507"	"GSM7963508"	"GSM7963509"	"GSM7963510"	"GSM7963511"	"GSM7963512"	"GSM7963513"	"GSM7963514"	"GSM7963515"	"GSM7963516"	"GSM7963517"	"GSM7963518"	"GSM7963519"	"GSM7963520"	"GSM7963521"	"GSM7963522"	"GSM7963523"	"GSM7963524"	"GSM7963525"	"GSM7963526"	"GSM7963527"	"GSM7963528"	"GSM7963529"	"GSM7963530"	"GSM7963531"	"GSM7963532"	"GSM7963533"	"GSM7963534"	"GSM7963535"	"GSM7963536"	"GSM7963537"	"GSM7963538"	"GSM7963539"	"GSM7963540"	"GSM7963541"	"GSM7963542"	"GSM7963543"	"GSM7963544"	"GSM7963545"	"GSM7963546"	"GSM7963547"	"GSM7963548"	"GSM7963549"	"GSM7963550"	"GSM7963551"	"GSM7963552"	"GSM7963553"	"GSM7963554"	"GSM7963555"	"GSM7963556"	"GSM7963557"	"GSM7963558"	"GSM7963559"	"GSM7963560"	"GSM7963561"	"GSM7963562"	"GSM7963563"	"GSM7963564"	"GSM7963565"	"GSM7963566"	"GSM7963567"	"GSM7963568"	"GSM7963569"	"GSM7963570"	"GSM7963571"	"GSM7963572"	"GSM7963573"	"GSM7963574"	"GSM7963575"	"GSM7963576"	"GSM7963577"	"GSM7963578"	"GSM7963579"	"GSM7963580"	"GSM7963581"	"GSM7963582"	"GSM7963583"	"GSM7963584"	"GSM7963585"	"GSM7963586"	"GSM7963587"	"GSM7963588"	"GSM7963589"	"GSM7963590"	"GSM7963591"	"GSM7963592"	"GSM7963593"	"GSM7963594"	"GSM7963595"	"GSM7963596"	"GSM7963597"	"GSM7963598"	"GSM7963599"	"GSM7963600"	"GSM7963601"	"GSM7963602"
species	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"	"Homo sapiens"
tissue	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"	"tissue: Dorsal root ganglia"
donor_id	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N2"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N3"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"	"donor id: N4"
age	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 61"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 56"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"	"age(years): 23"
race	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"	"race: Caucasian"
gender	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Female"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"	"gender: Male"
region	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: lumbar 5"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: lumbar 2"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: thoracic 11"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: lumbar 3"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"	"region: thoracic 12"
sequencing_batch	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 1"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 2"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"	"sequencing batch: 3"
dissected_soma_size	"dissected soma size: 43.35091233"	"dissected soma size: 46.81073909"	"dissected soma size: 63.04802873"	"dissected soma size: 64.90878003"	"dissected soma size: 44.02125191"	"dissected soma size: 81.64975426"	"dissected soma size: 75.54243666"	"dissected soma size: 76.72978402"	"dissected soma size: 66.02705715"	"dissected soma size: 89.43428289"	"dissected soma size: 77.00309514"	"dissected soma size: 62.48001689"	"dissected soma size: 58.88150252"	"dissected soma size: 96.53414104"	"dissected soma size: 59.61217686"	"dissected soma size: 59.62285526"	"dissected soma size: 58.07591249"	"dissected soma size: 66.01741464"	"dissected soma size: 80.61398605"	"dissected soma size: 72.98790548"	"dissected soma size: 63.82079107"	"dissected soma size: 47.83316319"	"dissected soma size: 68.44156898"	"dissected soma size: 60.22839467"	"dissected soma size: 41.56664638"	"dissected soma size: 96.25676249"	"dissected soma size: 66.62216695"	"dissected soma size: 63.2395879"	"dissected soma size: 76.71318841"	"dissected soma size: 53.0098078"	"dissected soma size: 75.95425252"	"dissected soma size: 72.91809398"	"dissected soma size: 68.38573629"	"dissected soma size: 62.04033534"	"dissected soma size: 63.13883973"	"dissected soma size: 51.99122618"	"dissected soma size: 48.11184399"	"dissected soma size: 59.61217686"	"dissected soma size: 75.3568077"	"dissected soma size: 89.44140091"	"dissected soma size: 87.44756621"	"dissected soma size: 78.30658818"	"dissected soma size: 64.8204487"	"dissected soma size: 73.8463612"	"dissected soma size: 49.53314693"	"dissected soma size: 83.30159445"	"dissected soma size: 47.793219"	"dissected soma size: 61.56650217"	"dissected soma size: 61.08899385"	"dissected soma size: 73.85498157"	"dissected soma size: 60.96381135"	"dissected soma size: 50.70178187"	"dissected soma size: 74.83119412"	"dissected soma size: 41.53600381"	"dissected soma size: 50.53828816"	"dissected soma size: 57.6909718"	"dissected soma size: 71.07893218"	"dissected soma size: 44.05016571"	"dissected soma size: 44.76694179"	"dissected soma size: 82.61091984"	"dissected soma size: 50.87726446"	"dissected soma size: 48.12507425"	"dissected soma size: 68.8773593"	"dissected soma size: 58.72994113"	"dissected soma size: 66.54567764"	"dissected soma size: 54.04453317"	"dissected soma size: 52.41804064"	"dissected soma size: 43.14482873"	"dissected soma size: 92.65772713"	"dissected soma size: 52.1867745"	"dissected soma size: 58.76245148"	"dissected soma size: 67.78732621"	"dissected soma size: 64.702486"	"dissected soma size: 47.6998858"	"dissected soma size: 65.52376529"	"dissected soma size: 57.67993574"	"dissected soma size: 87.15587905"	"dissected soma size: 69.95950896"	"dissected soma size: 71.58763339"	"dissected soma size: 84.46278051"	"dissected soma size: 76.80441987"	"dissected soma size: 53.17767461"	"dissected soma size: 55.34812008"	"dissected soma size: 47.04136867"	"dissected soma size: 60.98469295"	"dissected soma size: 60.76507832"	"dissected soma size: 43.59984716"	"dissected soma size: 99.38708446"	"dissected soma size: 52.84140771"	"dissected soma size: 62.22476614"	"dissected soma size: 73.70829824"	"dissected soma size: 68.10588624"	"dissected soma size: 93.13743161"	"dissected soma size: 40.40377332"	"dissected soma size: 64.24540854"	"dissected soma size: 54.83592788"	"dissected soma size: 78.26241897"	"dissected soma size: 60.20391247"	"dissected soma size: 47.68127898"	"dissected soma size: 51.7625235"	"dissected soma size: 73.39826104"	"dissected soma size: 70.52683655"	"dissected soma size: 54.88762977"	"dissected soma size: 66.67784062"	"dissected soma size: 64.00226575"	"dissected soma size: 78.26999882"	"dissected soma size: 56.00207564"	"dissected soma size: 85.76859205"	"dissected soma size: 58.82770126"	"dissected soma size: 43.98329679"	"cell identity: hNP2"	"dissected soma size: 36.58449956"	"dissected soma size: 68.79700905"	"dissected soma size: 52.18485459"	"dissected soma size: 40.52564145"	"dissected soma size: 58.40327464"	"dissected soma size: 48.30819562"	"cell identity: hPEP.NTRK3"	"dissected soma size: 57.85824394"	"cell identity: hPEP.0"	"dissected soma size: 52.83798943"	"dissected soma size: 62.1524082"	"dissected soma size: 40.78647033"	"dissected soma size: 43.9239161"	"dissected soma size: 79.62424787"	"dissected soma size: 52.92248006"	"dissected soma size: 42.51437983"	"dissected soma size: 45.45834889"	"dissected soma size: 54.13226331"	"dissected soma size: 52.00585424"	"dissected soma size: 53.55139165"	"dissected soma size: 46.42640754"	"dissected soma size: 63.60798172"	"dissected soma size: 60.65641722"	"dissected soma size: 51.36848725"	"dissected soma size: 65.43626412"	"dissected soma size: 54.87450751"	"dissected soma size: 57.48092444"	"dissected soma size: 59.43035047"	"dissected soma size: 40.30699303"	"dissected soma size: 56.7564631"	"dissected soma size: 67.76854076"	"dissected soma size: 63.78087813"	"dissected soma size: 68.41365833"	"dissected soma size: 57.0919722"	"dissected soma size: 55.79489887"	"dissected soma size: 50.47526496"	"dissected soma size: 52.7570061"	"dissected soma size: 51.75805298"	"dissected soma size: 52.6120026"	"dissected soma size: 51.12690906"	"dissected soma size: 56.04535673"	"dissected soma size: 48.49405612"	"dissected soma size: 40.40164754"	"dissected soma size: 49.99369229"	"dissected soma size: 47.85977413"	"dissected soma size: 51.05214386"	"dissected soma size: 42.23515873"	"dissected soma size: 55.02511871"	"dissected soma size: 55.79489887"	"dissected soma size: 53.72554418"	"dissected soma size: 77.54682761"	"dissected soma size: 61.46301159"	"dissected soma size: 45.70980342"	"dissected soma size: 51.94222407"	"dissected soma size: 55.52038356"	"dissected soma size: 49.05530329"	"dissected soma size: 43.48287889"	"dissected soma size: 69.11725438"	"dissected soma size: 63.31001623"	"dissected soma size: 63.07831359"	"dissected soma size: 49.80231596"	"dissected soma size: 42.49063079"	"dissected soma size: 50.03187971"	"dissected soma size: 48.88630346"	"dissected soma size: 56.1134694"	"dissected soma size: 46.53794691"	"dissected soma size: 53.47612613"	"dissected soma size: 61.46301159"	"dissected soma size: 45.50041247"	"dissected soma size: 57.83424886"	"dissected soma size: 60.18609945"	"dissected soma size: 52.43018428"	"dissected soma size: 44.62450764"	"dissected soma size: 57.1588377"	"dissected soma size: 57.50307082"	"dissected soma size: 56.19282983"	"dissected soma size: 47.85977413"	"dissected soma size: 42.90809263"	"dissected soma size: 47.51266938"	"dissected soma size: 55.63492955"	"dissected soma size: 62.63266803"	"dissected soma size: 79.15561829"	"dissected soma size: 49.27542849"	"dissected soma size: 68.27393392"	"dissected soma size: 53.28530973"	"dissected soma size: 52.40589418"	"dissected soma size: 65.88227138"	"dissected soma size: 41.82620189"	"dissected soma size: 49.94273031"	"dissected soma size: 53.12976679"	"dissected soma size: 52.55146639"	"dissected soma size: 48.45465675"	"dissected soma size: 60.92202669"	"dissected soma size: 60.43942684"	"dissected soma size: 50.87726446"	"dissected soma size: 65.27066408"	"dissected soma size: 51.8440809"	"dissected soma size: 65.27066408"	"dissected soma size: 50.96477917"	"dissected soma size: 54.43186999"	"dissected soma size: 46.89226729"	"dissected soma size: 42.34053979"	"dissected soma size: 49.89171627"	"dissected soma size: 43.18907229"	"dissected soma size: 54.95565714"	"dissected soma size: 37.50906185"	"dissected soma size: 77.20125913"	"dissected soma size: 46.31855643"	"dissected soma size: 50.88977577"	"dissected soma size: 50.13357063"	"dissected soma size: 45.77938757"	"dissected soma size: 36.89299522"	"dissected soma size: 52.2964491"	"dissected soma size: 50.78959895"	"dissected soma size: 47.4053566"	"dissected soma size: 47.89966283"	"dissected soma size: 78.42844092"	"dissected soma size: 42.71477846"	"dissected soma size: 49.73836022"	"dissected soma size: 85.42210461"	"dissected soma size: 45.00804539"	"dissected soma size: 55.57768606"	"dissected soma size: 52.35728017"	"dissected soma size: 61.53547327"	"dissected soma size: 44.13679346"	"dissected soma size: 44.69578145"	"dissected soma size: 45.12106012"	"dissected soma size: 111.0636976"	"dissected soma size: 70.74676484"	"dissected soma size: 47.95279613"	"dissected soma size: 51.27611242"	"dissected soma size: 48.78201256"	"dissected soma size: 45.72372872"	"dissected soma size: 45.31815774"	"dissected soma size: 45.73764978"	"dissected soma size: 39.20204276"	"dissected soma size: 38.90863458"	"dissected soma size: 53.91480251"	"dissected soma size: 46.63360569"	"dissected soma size: 60.32345051"	"dissected soma size: 49.36578299"	"dissected soma size: 71.02517274"	"dissected soma size: 49.17196291"	"dissected soma size: 65.44599225"	"dissected soma size: 39.63808301"	"dissected soma size: 71.20421303"	"dissected soma size: 51.92996632"	"dissected soma size: 44.18004363"	"dissected soma size: 45.06458819"	"dissected soma size: 54.12692715"	"dissected soma size: 63.32007103"	"dissected soma size: 50.98975578"	"dissected soma size: 51.80722928"	"dissected soma size: 92.28596567"	"dissected soma size: 45.57031636"	"dissected soma size: 54.3382237"	"dissected soma size: 43.9488842"	"dissected soma size: 46.55162448"	"dissected soma size: 61.61818228"	"dissected soma size: 56.98035534"	"dissected soma size: 57.0250283"	"dissected soma size: 48.78201256"	"dissected soma size: 53.20161235"	"dissected soma size: 58.45831845"	"dissected soma size: 51.31334547"	"dissected soma size: 45.61220727"	"dissected soma size: 75.74442215"	"dissected soma size: 52.25991648"	"dissected soma size: 62.16335001"	"dissected soma size: 58.29473735"	"dissected soma size: 48.71671738"	"dissected soma size: 56.73402528"	"dissected soma size: 42.22008282"	"dissected soma size: 55.35962099"	"dissected soma size: 45.90437324"	"dissected soma size: 35.48568629"	"dissected soma size: 70.61165727"	"dissected soma size: 61.21392035"	"dissected soma size: 55.35962099"	"dissected soma size: 55.81771421"	"dissected soma size: 68.18062523"	"dissected soma size: 68.67371662"	"dissected soma size: 68.8773593"	"dissected soma size: 64.87934962"	"dissected soma size: 58.4365341"	"dissected soma size: 50.28572142"	"dissected soma size: 48.24398333"	"dissected soma size: 42.86355905"	"dissected soma size: 56.4302415"	"dissected soma size: 58.01010407"	"dissected soma size: 63.12875607"	"dissected soma size: 50.13357063"	"dissected soma size: 44.79537428"	"dissected soma size: 47.52606644"	"dissected soma size: 47.93951833"	"dissected soma size: 48.45465675"	"dissected soma size: 48.54653888"	"dissected soma size: 54.7932377"	"dissected soma size: 52.27209685"	"dissected soma size: 54.7699956"	"dissected soma size: 62.24522472"	"dissected soma size: 52.84140771"	"dissected soma size: 47.21697238"	"dissected soma size: 51.91770567"	"dissected soma size: 42.12951407"	"dissected soma size: 57.9222432"	"dissected soma size: 56.76767868"	"dissected soma size: 43.97784558"	"dissected soma size: 49.82787528"	"dissected soma size: 46.18090811"	"dissected soma size: 55.81771421"	"dissected soma size: 55.7834877"	"dissected soma size: 64.30771425"	"dissected soma size: 42.59537982"	"dissected soma size: 61.0264347"	"dissected soma size: 61.05772228"	"dissected soma size: 49.71275489"	"dissected soma size: 50.43741323"	"dissected soma size: 69.43888131"	"dissected soma size: 51.23885231"	"dissected soma size: 44.36697386"	"dissected soma size: 56.7003519"	"dissected soma size: 48.84722047"	"dissected soma size: 50.71433648"	"dissected soma size: 45.0787128"	"dissected soma size: 43.96336727"	"dissected soma size: 37.27069046"	"dissected soma size: 47.97934072"	"dissected soma size: 55.26754672"	"dissected soma size: 55.24450417"	"dissected soma size: 54.46694585"	"dissected soma size: 40.79367737"	"dissected soma size: 64.39674889"	"dissected soma size: 54.60702395"	"dissected soma size: 47.36505163"	"dissected soma size: 44.4816174"	"dissected soma size: 47.14950989"	"dissected soma size: 50.51308832"	"dissected soma size: 56.7901032"	"dissected soma size: 44.35262259"	"dissected soma size: 48.40207436"	"dissected soma size: 38.46432602"	"dissected soma size: 50.08275098"	"dissected soma size: 63.35022584"	"dissected soma size: 53.36887586"	"dissected soma size: 56.65542293"	"dissected soma size: 46.96009933"	"dissected soma size: 55.34812008"	"dissected soma size: 65.52203054"	"dissected soma size: 59.0670574"	"cell identity: hPEP.0"	"dissected soma size: 53.50047756"	"dissected soma size: 45.6731716"	"dissected soma size: 41.28792328"	"dissected soma size: 47.72441733"	"dissected soma size: 42.78215126"	"dissected soma size: 54.058924"	"dissected soma size: 56.39562383"	"dissected soma size: 65.0495111"	"dissected soma size: 51.78257622"	"dissected soma size: 99.92369429"	"dissected soma size: 55.88610432"	"dissected soma size: 69.00663721"	"dissected soma size: 68.35780284"	"dissected soma size: 55.90888244"	"dissected soma size: 85.13095566"	"dissected soma size: 92.3135549"	"dissected soma size: 54.46694585"	"dissected soma size: 69.23689022"	"dissected soma size: 69.5762663"	"dissected soma size: 66.94627155"	"dissected soma size: 43.35091233"	"dissected soma size: 82.56466943"	"dissected soma size: 86.84855319"	"dissected soma size: 78.18454552"	"dissected soma size: 56.32861617"	"dissected soma size: 52.47873076"	"dissected soma size: 62.33720524"	"dissected soma size: 63.97024282"	"dissected soma size: 46.29105951"	"dissected soma size: 54.23267833"	"dissected soma size: 76.43051178"	"dissected soma size: 57.51414081"	"dissected soma size: 47.59299516"	"dissected soma size: 54.12692715"	"dissected soma size: 86.96575777"	"dissected soma size: 74.3361266"	"dissected soma size: 72.58557244"	"dissected soma size: 53.8439086"	"dissected soma size: 69.64942748"	"dissected soma size: 81.89887689"	"dissected soma size: 92.80875857"	"dissected soma size: 42.68496011"	"dissected soma size: 57.37006431"	"dissected soma size: 55.65781046"	"dissected soma size: 52.35728017"	"dissected soma size: 60.03783151"	"dissected soma size: 44.46730312"	"dissected soma size: 45.91823954"	"dissected soma size: 42.80410887"	"dissected soma size: 43.78925424"	"dissected soma size: 46.70181343"	"dissected soma size: 46.70181343"	"dissected soma size: 57.21449926"	"dissected soma size: 41.35167152"	"dissected soma size: 49.78953139"	"dissected soma size: 52.79319474"	"dissected soma size: 71.8627837"	"dissected soma size: 55.40560075"	"dissected soma size: 48.84722047"	"dissected soma size: 61.64916955"	"dissected soma size: 54.24441573"	"dissected soma size: 46.15332918"	"dissected soma size: 46.76992169"	"dissected soma size: 62.72408039"	"dissected soma size: 45.80719164"	"dissected soma size: 41.44394015"	"dissected soma size: 50.00642467"	"dissected soma size: 39.84632655"	"dissected soma size: 46.04284828"	"dissected soma size: 47.39192542"	"dissected soma size: 54.7699956"	"dissected soma size: 49.41733996"	"dissected soma size: 57.05851007"	"dissected soma size: 55.34812008"	"dissected soma size: 34.35539373"	"dissected soma size: 50.70178187"	"dissected soma size: 50.53828816"	"dissected soma size: 48.79506112"	"dissected soma size: 53.52372394"	"dissected soma size: 52.46659835"	"dissected soma size: 52.19897195"	"dissected soma size: 61.89650923"	"dissected soma size: 77.94805457"	"dissected soma size: 61.93763649"	"dissected soma size: 65.1241967"	"dissected soma size: 47.80653744"	"dissected soma size: 94.44742666"	"dissected soma size: 59.80409637"	"dissected soma size: 63.72096184"	"dissected soma size: 52.41804064"	"dissected soma size: 45.82108734"	"dissected soma size: 41.73477839"	"dissected soma size: 62.86601319"	"dissected soma size: 38.28183304"	"dissected soma size: 40.6059754"	"dissected soma size: 60.32345051"	"dissected soma size: 71.97785629"	"dissected soma size: 41.38245059"	"dissected soma size: 56.27207835"	"dissected soma size: 50.81466167"	"dissected soma size: 44.30954086"	"dissected soma size: 63.78087813"	"dissected soma size: 49.58452987"	"dissected soma size: 80.03541878"	"dissected soma size: 49.39769587"	"dissected soma size: 44.5308789"	"dissected soma size: 45.18510219"	"dissected soma size: 37.8033428"	"dissected soma size: 69.97847411"	"dissected soma size: 47.02433075"	"dissected soma size: 55.17398028"	"dissected soma size: 59.91045266"	"dissected soma size: 50.56347545"	"dissected soma size: 50.5886502"	"dissected soma size: 46.70181343"	"dissected soma size: 55.3251111"	"dissected soma size: 49.89171627"	"dissected soma size: 51.54853029"	"dissected soma size: 64.85972192"	"dissected soma size: 56.03399656"	"dissected soma size: 49.78953139"	"dissected soma size: 54.87450751"	"dissected soma size: 49.40445576"	"dissected soma size: 59.48388652"	"dissected soma size: 44.12236731"	"dissected soma size: 50.27305978"	"dissected soma size: 56.94682762"	"dissected soma size: 48.13830087"	"dissected soma size: 62.31677686"	"dissected soma size: 49.02934123"	"dissected soma size: 54.83969234"	"dissected soma size: 47.61974032"	"dissected soma size: 45.82108734"	"dissected soma size: 49.81509726"	"dissected soma size: 45.03632566"	"dissected soma size: 40.3701207"	"dissected soma size: 56.52042173"	"dissected soma size: 54.43186999"	"dissected soma size: 66.86063205"	"dissected soma size: 57.19224114"	"dissected soma size: 50.5886502"	"dissected soma size: 56.53168414"	"dissected soma size: 51.99122618"	"dissected soma size: 48.70364783"	"dissected soma size: 42.0387502"	"dissected soma size: 49.64868378"	"dissected soma size: 55.31360301"	"dissected soma size: 66.16190467"	"dissected soma size: 47.793219"	"dissected soma size: 58.31657468"	"dissected soma size: 47.48586392"	"dissected soma size: 55.71497162"	"dissected soma size: 44.19445095"	"dissected soma size: 54.89770536"	"dissected soma size: 41.59726639"	"dissected soma size: 41.53600381"	"dissected soma size: 49.08125161"	"dissected soma size: 45.47242083"	"dissected soma size: 44.46730312"	"dissected soma size: 50.85223259"	"dissected soma size: 54.39677153"	"dissected soma size: 40.63731927"	"dissected soma size: 57.10312189"	"dissected soma size: 32.95562699"	"dissected soma size: 42.4306581"	"dissected soma size: 61.13066438"	"dissected soma size: 59.51598503"	"dissected soma size: 49.4302208"	"dissected soma size: 51.5361789"	"dissected soma size: 53.03382131"	"dissected soma size: 41.94778994"	"dissected soma size: 48.46779343"	"dissected soma size: 56.41895884"	"dissected soma size: 34.02021114"	"dissected soma size: 47.57961695"	"dissected soma size: 47.12249785"	"dissected soma size: 55.74924019"	"dissected soma size: 41.82620189"	"dissected soma size: 39.46101861"	"dissected soma size: 46.55162448"	"dissected soma size: 43.43893458"	"dissected soma size: 50.03187971"	"dissected soma size: 48.96437586"	"dissected soma size: 49.67432214"	"dissected soma size: 72.13688452"	"dissected soma size: 70.20477449"	"dissected soma size: 81.93773388"	"dissected soma size: 42.90809263"	"dissected soma size: 49.19784971"	"dissected soma size: 46.91941187"	"dissected soma size: 45.93210165"	"dissected soma size: 62.99752161"	"dissected soma size: 48.17795896"	"dissected soma size: 68.5345224"	"dissected soma size: 48.25717736"	"dissected soma size: 53.21357718"	"dissected soma size: 39.18579994"	"dissected soma size: 51.52382455"	"dissected soma size: 57.80121644"	"dissected soma size: 44.15121489"	"dissected soma size: 33.73834869"	"dissected soma size: 42.56547783"	"dissected soma size: 48.84722047"	"dissected soma size: 52.34511961"	"dissected soma size: 33.37790618"	"dissected soma size: 35.30582863"	"dissected soma size: 68.35780284"	"dissected soma size: 40.5117983"	"dissected soma size: 56.40767392"	"dissected soma size: 38.84313178"	"dissected soma size: 40.32278421"	"dissected soma size: 39.23450823"	"dissected soma size: 49.57168913"	"dissected soma size: 50.24772692"	"dissected soma size: 46.90584154"	"dissected soma size: 52.27209685"	"dissected soma size: 66.66037869"	"dissected soma size: 58.42563887"	"dissected soma size: 54.82808237"	"dissected soma size: 51.9667309"	"dissected soma size: 31.39247401"	"dissected soma size: 66.20999794"	"dissected soma size: 60.7231569"	"dissected soma size: 82.31756175"	"dissected soma size: 79.52471897"	"dissected soma size: 47.76657098"	"dissected soma size: 78.99460181"	"dissected soma size: 82.07746674"	"dissected soma size: 42.41565168"	"dissected soma size: 49.3270799"	"dissected soma size: 52.79319474"	"dissected soma size: 53.10579667"	"dissected soma size: 57.06966629"	"dissected soma size: 51.01472017"	"dissected soma size: 53.83208387"	"dissected soma size: 35.78936998"	"dissected soma size: 50.42478968"	"dissected soma size: 63.50078509"	"dissected soma size: 52.64829091"	"dissected soma size: 35.70031941"	"dissected soma size: 31.31125143"	"dissected soma size: 32.37091539"	"dissected soma size: 42.59537982"	"dissected soma size: 39.10448464"	"dissected soma size: 59.05423924"	"dissected soma size: 44.79537428"	"dissected soma size: 46.56529804"	"dissected soma size: 48.04563803"	"dissected soma size: 49.68713636"	"dissected soma size: 48.13830087"	"dissected soma size: 49.75115795"	"dissected soma size: 36.19618093"	"dissected soma size: 59.70821372"	"dissected soma size: 63.17915828"	"dissected soma size: 73.37068139"	"dissected soma size: 62.60216758"	"dissected soma size: 64.9283929"	"dissected soma size: 55.27906439"	"dissected soma size: 55.30209253"	"dissected soma size: 48.49405612"	"dissected soma size: 44.25203332"	"dissected soma size: 55.52038356"	"dissected soma size: 65.94987763"	"dissected soma size: 38.89226922"	"dissected soma size: 37.98131684"	"dissected soma size: 79.11539489"	"dissected soma size: 43.32153184"	"dissected soma size: 44.66728557"	"dissected soma size: 43.90540631"	"dissected soma size: 56.39638674"	"dissected soma size: 69.05274923"	"dissected soma size: 53.68998397"	"dissected soma size: 47.71323029"	"dissected soma size: 44.95143148"	"dissected soma size: 45.77938757"	"dissected soma size: 49.66150461"	"dissected soma size: 48.58586375"	"dissected soma size: 44.56740652"	"dissected soma size: 58.09783207"	"dissected soma size: 48.88630346"	"dissected soma size: 50.81466167"	"dissected soma size: 52.21116654"	"dissected soma size: 44.0646155"	"dissected soma size: 51.20156509"	"dissected soma size: 67.42006565"	"dissected soma size: 50.41216296"	"dissected soma size: 50.83971204"	"dissected soma size: 39.2993587"	"dissected soma size: 44.69578145"	"dissected soma size: 52.34511961"	"dissected soma size: 60.02722693"	"dissected soma size: 49.14606248"	"dissected soma size: 42.56547783"	"dissected soma size: 42.83384427"	"dissected soma size: 42.96739894"	"dissected soma size: 50.48787589"	"dissected soma size: 60.60772351"	"dissected soma size: 39.76636193"	"dissected soma size: 42.80410887"	"dissected soma size: 51.36294756"	"dissected soma size: 40.79367737"	"dissected soma size: 50.62638884"	"dissected soma size: 38.46432602"	"dissected soma size: 70.18663607"	"dissected soma size: 72.16335515"	"dissected soma size: 43.93439635"	"dissected soma size: 55.84052022"	"dissected soma size: 68.03106514"	"dissected soma size: 49.80231596"	"dissected soma size: 44.58168865"	"dissected soma size: 43.61444614"	"dissected soma size: 42.35557282"	"dissected soma size: 42.50561074"	"dissected soma size: 65.04594591"	"dissected soma size: 37.08232371"	"dissected soma size: 41.73477839"	"dissected soma size: 42.96739894"	"dissected soma size: 36.40662562"	"dissected soma size: 39.10448464"	"dissected soma size: 48.4415165"	"dissected soma size: 41.22832547"	"dissected soma size: 43.93439635"	"dissected soma size: 68.96049435"	"dissected soma size: 45.89050275"	"dissected soma size: 44.35262259"	"dissected soma size: 51.7088299"	"dissected soma size: 43.91990372"	"dissected soma size: 45.00804539"	"dissected soma size: 57.75714384"	"dissected soma size: 40.54321497"	"dissected soma size: 44.09350086"	"dissected soma size: 49.3399843"	"dissected soma size: 63.31001623"	"dissected soma size: 36.28401455"	"dissected soma size: 49.3270799"	"dissected soma size: 59.39820569"	"dissected soma size: 50.78959895"	"dissected soma size: 44.71002258"	"dissected soma size: 61.04729487"	"dissected soma size: 53.22553931"	"dissected soma size: 58.76245148"	"dissected soma size: 56.48662101"	"dissected soma size: 35.19747289"	"dissected soma size: 41.84141972"	"dissected soma size: 56.90209326"	"dissected soma size: 56.64418512"	"dissected soma size: 43.43893458"	"dissected soma size: 43.49751712"	"dissected soma size: 62.38824693"	"dissected soma size: 49.22372289"	"dissected soma size: 45.12106012"	"dissected soma size: 48.70364783"	"dissected soma size: 40.84046807"	"dissected soma size: 39.75034971"	"dissected soma size: 45.12106012"	"dissected soma size: 49.52029286"	"dissected soma size: 38.56350365"	"dissected soma size: 44.45298423"	"dissected soma size: 52.55146639"	"dissected soma size: 44.3813205"	"dissected soma size: 36.00219269"	"dissected soma size: 49.40445576"	"dissected soma size: 45.77938757"	"dissected soma size: 48.33626593"	"dissected soma size: 31.9153827"	"dissected soma size: 39.86230022"	"dissected soma size: 32.39057583"	"dissected soma size: 45.79329171"	"dissected soma size: 45.51440184"	"dissected soma size: 58.78411506"	"dissected soma size: 69.40219939"	"dissected soma size: 36.42410779"	"dissected soma size: 50.38690004"	"dissected soma size: 64.38686223"	"dissected soma size: 44.33826666"	"dissected soma size: 44.13679346"	"dissected soma size: 46.64725521"	"dissected soma size: 43.83284734"	"dissected soma size: 29.9392828"	"dissected soma size: 41.62786386"	"dissected soma size: 43.36559511"	"dissected soma size: 44.66728557"	"dissected soma size: 47.37849043"	"dissected soma size: 46.78353145"	"dissected soma size: 42.67004312"	"dissected soma size: 43.61444614"	"dissected soma size: 37.52603044"	"dissected soma size: 41.87183878"	"dissected soma size: 55.10604655"	"dissected soma size: 41.87183878"	"dissected soma size: 49.08125161"	"dissected soma size: 35.71814729"	"dissected soma size: 69.42054277"	"dissected soma size: 78.52578698"	"dissected soma size: 55.79489887"	"dissected soma size: 60.58671199"	"dissected soma size: 59.61217686"	"dissected soma size: 64.32751037"	"dissected soma size: 51.12690906"	"dissected soma size: 57.97717185"	"dissected soma size: 68.16194817"	"dissected soma size: 66.11377641"	"dissected soma size: 67.63689646"	"dissected soma size: 45.72372872"	"dissected soma size: 72.17217654"	"dissected soma size: 108.58268"	"dissected soma size: 86.20107952"	"dissected soma size: 56.81251888"	"dissected soma size: 76.20528632"	"dissected soma size: 61.40083351"	"dissected soma size: 81.16490323"	"dissected soma size: 83.76648047"	"dissected soma size: 74.31042995"	"dissected soma size: 82.78028356"	"dissected soma size: 46.27730492"	"dissected soma size: 83.05667927"	"dissected soma size: 93.26721138"	"dissected soma size: 46.04284828"	"dissected soma size: 63.1186708"	"dissected soma size: 46.46949865"	"dissected soma size: 74.44737627"	"dissected soma size: 54.64198737"	"dissected soma size: 59.33386387"	"dissected soma size: 64.14912532"	"dissected soma size: 33.4921492"	"dissected soma size: 55.72639682"	"dissected soma size: 62.89638569"	"dissected soma size: 60.77555415"	"dissected soma size: 81.57955144"	"dissected soma size: 38.81033892"	"dissected soma size: 80.84267867"	"dissected soma size: 57.28122173"	"dissected soma size: 57.1588377"	"dissected soma size: 76.71318841"	"dissected soma size: 50.39953308"	"dissected soma size: 75.58456148"	"dissected soma size: 50.90228401"	"dissected soma size: 65.42653454"	"dissected soma size: 55.74924019"	"dissected soma size: 45.77938757"	"dissected soma size: 57.63577039"	"dissected soma size: 48.46779343"	"dissected soma size: 86.2675215"	"dissected soma size: 43.7019376"	"dissected soma size: 74.97568063"	"dissected soma size: 52.16237105"	"dissected soma size: 72.37477168"	"dissected soma size: 91.84341091"	"dissected soma size: 43.29213141"	"dissected soma size: 52.97376713"	"dissected soma size: 48.72978341"	"dissected soma size: 74.92471721"	"dissected soma size: 69.40219939"	"dissected soma size: 81.82110755"	"dissected soma size: 46.23601661"	"dissected soma size: 69.36549806"	"dissected soma size: 68.06848597"	"dissected soma size: 44.5388285"	"dissected soma size: 65.68872727"	"dissected soma size: 45.27599477"	"dissected soma size: 64.96760087"	"dissected soma size: 53.85573073"	"dissected soma size: 56.80131215"	"dissected soma size: 63.17915828"	"dissected soma size: 53.32113982"	"dissected soma size: 58.84905794"	"dissected soma size: 62.86601319"	"dissected soma size: 52.3694379"	"dissected soma size: 64.00009129"	"dissected soma size: 57.95520664"	"dissected soma size: 53.79659408"	"dissected soma size: 44.42433261"	"dissected soma size: 55.30209253"	"dissected soma size: 68.49735617"	"dissected soma size: 65.06551743"	"dissected soma size: 55.99990225"	"dissected soma size: 49.54599766"	"dissected soma size: 48.17795896"	"dissected soma size: 59.43035047"	"dissected soma size: 55.10604655"	"dissected soma size: 60.14377448"	"dissected soma size: 49.55884506"	"dissected soma size: 55.12914697"	"dissected soma size: 60.67071441"	"dissected soma size: 75.43280203"	"dissected soma size: 47.63310726"	"dissected soma size: 44.99389859"	"dissected soma size: 43.97784558"	"dissected soma size: 48.32309349"	"dissected soma size: 57.50307082"	"dissected soma size: 60.23896384"	"dissected soma size: 62.08136731"	"dissected soma size: 58.93553713"	"dissected soma size: 56.23812837"	"dissected soma size: 71.82733966"	"dissected soma size: 52.78113462"	"dissected soma size: 71.3917221"	"dissected soma size: 54.66528389"	"dissected soma size: 56.7564631"	"dissected soma size: 47.18999896"	"dissected soma size: 62.58182569"	"dissected soma size: 83.94109697"	"dissected soma size: 60.56569318"	"dissected soma size: 46.7426903"	"dissected soma size: 46.63360569"	"dissected soma size: 65.05573241"	"dissected soma size: 55.10604655"	"dissected soma size: 54.3147869"	"dissected soma size: 74.29329391"	"dissected soma size: 42.61032295"	"dissected soma size: 44.0646155"	"dissected soma size: 44.56740652"	"cell identity: Glia.cont"	"dissected soma size: 45.00804539"	"dissected soma size: 45.27599477"	"dissected soma size: 44.85218518"	"dissected soma size: 48.24398333"	"dissected soma size: 60.08023112"	"dissected soma size: 46.56529804"	"dissected soma size: 75.77803405"	"dissected soma size: 45.37431409"	"dissected soma size: 55.41708973"	"dissected soma size: 55.90888244"	"dissected soma size: 57.19224114"	"dissected soma size: 61.57684166"	"dissected soma size: 54.07986025"	"dissected soma size: 55.46302185"	"dissected soma size: 80.86629961"	"dissected soma size: 59.10811594"	"dissected soma size: 39.81435998"	"dissected soma size: 57.85625999"	"dissected soma size: 45.8627492"	"dissected soma size: 52.7449377"	"dissected soma size: 50.55088337"	"dissected soma size: 67.18358626"	"dissected soma size: 36.35412865"	"dissected soma size: 51.41250179"	"dissected soma size: 91.17555116"	"dissected soma size: 48.34943478"	"dissected soma size: 37.84698815"	"dissected soma size: 52.01570992"	"dissected soma size: 57.03619107"	"dissected soma size: 54.54870179"	"dissected soma size: 54.90930061"	"dissected soma size: 49.08125161"	"dissected soma size: 55.96578717"	"dissected soma size: 81.12567608"	"dissected soma size: 52.7449377"	"dissected soma size: 52.91364481"	"dissected soma size: 87.85430127"	"dissected soma size: 38.24855895"	"dissected soma size: 44.92309777"	"dissected soma size: 52.13795618"	"dissected soma size: 55.48597365"	"dissected soma size: 50.86475006"	"dissected soma size: 66.93676146"	"dissected soma size: 71.71202555"	"dissected soma size: 71.79187813"	"dissected soma size: 56.47534961"	"dissected soma size: 54.70021007"	"dissected soma size: 49.59736729"	"dissected soma size: 72.93555312"	"dissected soma size: 51.81951607"	"dissected soma size: 48.07213136"	"dissected soma size: 39.39643425"	"dissected soma size: 50.38690004"	"dissected soma size: 60.43942684"	"dissected soma size: 31.37218808"	"dissected soma size: 64.23837977"	"dissected soma size: 54.7699956"	"dissected soma size: 50.32368724"	"dissected soma size: 37.04797231"	"dissected soma size: 39.75034971"	"dissected soma size: 56.35121541"	"dissected soma size: 53.05782394"	"dissected soma size: 38.31507823"	"dissected soma size: 47.95279613"	"dissected soma size: 49.45597241"	"dissected soma size: 42.87840872"	"dissected soma size: 54.00918299"	"dissected soma size: 46.31855643"	"dissected soma size: 69.39302588"	"dissected soma size: 44.32390609"	"dissected soma size: 47.41878398"	"dissected soma size: 42.90809263"	"dissected soma size: 41.59726639"	"dissected soma size: 49.31417212"	"dissected soma size: 52.6120026"	"dissected soma size: 52.41804064"	"dissected soma size: 60.19667604"	"dissected soma size: 57.76816514"	"dissected soma size: 53.82025654"	"dissected soma size: 46.08430966"	"dissected soma size: 44.68153579"	"dissected soma size: 57.94422091"	"dissected soma size: 54.97882075"	"dissected soma size: 39.33174384"	"dissected soma size: 44.79537428"	"dissected soma size: 48.21758443"	"dissected soma size: 58.46920759"	"dissected soma size: 59.09734453"	"dissected soma size: 53.18964483"	"dissected soma size: 45.16336774"	"dissected soma size: 56.12481347"	"dissected soma size: 55.95441085"	"dissected soma size: 55.38261564"	"dissected soma size: 64.48566061"	"dissected soma size: 54.60702395"	"dissected soma size: 55.97716117"	"dissected soma size: 69.43888131"	"dissected soma size: 43.49751712"	"dissected soma size: 57.06966629"	"dissected soma size: 38.03156778"	"dissected soma size: 47.97934072"	"dissected soma size: 51.54853029"	"dissected soma size: 52.33295623"	"dissected soma size: 52.59990093"	"dissected soma size: 46.33229877"	"dissected soma size: 46.64725521"	"dissected soma size: 47.73990808"	"dissected soma size: 39.34792641"	"dissected soma size: 51.49910695"	"dissected soma size: 65.59174106"	"dissected soma size: 57.55839949"	"dissected soma size: 54.10339881"	"dissected soma size: 37.57689023"	"dissected soma size: 56.20415789"	"dissected soma size: 53.73739235"	"cell identity: hPEP.NTRK3"	"dissected soma size: 42.52058542"	"dissected soma size: 42.96739894"	"dissected soma size: 42.84870424"	"dissected soma size: 59.47318316"	"dissected soma size: 46.1946914"	"dissected soma size: 66.2868746"	"dissected soma size: 68.25528239"	"dissected soma size: 57.08082034"	"dissected soma size: 52.9497264"	"dissected soma size: 44.33826666"	"dissected soma size: 52.39374491"	"dissected soma size: 55.7834877"	"dissected soma size: 69.26446915"	"dissected soma size: 68.72931537"	"dissected soma size: 56.4302415"	"dissected soma size: 55.35962099"	"dissected soma size: 51.437261"	"dissected soma size: 42.64019349"	"dissected soma size: 58.78411506"	"dissected soma size: 52.56357921"	"dissected soma size: 34.42943561"	"dissected soma size: 80.1943455"	"dissected soma size: 39.49327119"	"dissected soma size: 48.4415165"	"dissected soma size: 74.58407113"	"dissected soma size: 50.56347545"	"dissected soma size: 34.99794941"	"dissected soma size: 37.88061498"	"dissected soma size: 43.84736874"	"dissected soma size: 64.96760087"	"dissected soma size: 62.19406566"	"dissected soma size: 41.21288126"	"dissected soma size: 52.46659835"	"dissected soma size: 62.88626315"	"dissected soma size: 49.27542849"	"dissected soma size: 51.38773064"	"dissected soma size: 41.75002954"	"dissected soma size: 52.47873076"	"dissected soma size: 55.92026802"	"dissected soma size: 53.0098078"	"dissected soma size: 49.73836022"	"dissected soma size: 43.7456177"	"dissected soma size: 57.9332331"	"dissected soma size: 41.81097852"	"dissected soma size: 47.2439304"	"dissected soma size: 48.97737583"	"dissected soma size: 47.18999896"	"dissected soma size: 68.17128734"	"dissected soma size: 69.6128565"	"dissected soma size: 46.30481001"	"dissected soma size: 49.86618968"	"dissected soma size: 60.70218534"	"dissected soma size: 82.79566309"	"dissected soma size: 42.72967983"	"dissected soma size: 32.87826625"	"dissected soma size: 70.15941964"	"dissected soma size: 74.84820701"	"dissected soma size: 64.50540212"	"dissected soma size: 50.977269"	"dissected soma size: 37.94777925"	"dissected soma size: 53.34501318"	"dissected soma size: 55.55477215"	"dissected soma size: 49.66150461"	"dissected soma size: 54.70021007"	"dissected soma size: 64.06968364"	"dissected soma size: 52.15016504"	"dissected soma size: 31.69520497"	"dissected soma size: 42.71477846"	"dissected soma size: 48.86025161"	"dissected soma size: 68.16194817"	"dissected soma size: 58.95713711"	"dissected soma size: 70.80972682"	"dissected soma size: 86.31916324"	"dissected soma size: 62.72408039"	"dissected soma size: 51.32575049"	"dissected soma size: 68.002986"	"dissected soma size: 79.79643521"	"dissected soma size: 40.49608082"	"dissected soma size: 68.4322667"	"dissected soma size: 47.14950989"	"dissected soma size: 49.1590144"	"dissected soma size: 49.08125161"	"dissected soma size: 49.26250718"	"dissected soma size: 43.43893458"	"dissected soma size: 44.35262259"	"dissected soma size: 59.96356006"	"dissected soma size: 77.52219523"	"dissected soma size: 63.15900222"	"dissected soma size: 67.17410977"	"dissected soma size: 47.00075157"	"dissected soma size: 35.50362194"	"dissected soma size: 48.19117107"	"dissected soma size: 34.31831288"	"dissected soma size: 41.56664638"	"dissected soma size: 48.58586375"	"dissected soma size: 54.03275234"	"dissected soma size: 43.68736786"	"dissected soma size: 34.65061199"	"dissected soma size: 45.83497883"	"dissected soma size: 34.77898199"	"dissected soma size: 55.41708973"	"dissected soma size: 32.351243"	"dissected soma size: 64.80080316"	"dissected soma size: 76.93692705"	"dissected soma size: 43.0266235"	"dissected soma size: 60.71267203"	"dissected soma size: 47.27087305"	"dissected soma size: 56.55420224"	"dissected soma size: 37.1680633"	"dissected soma size: 55.20992232"	"dissected soma size: 44.18004363"	"dissected soma size: 46.34603703"	"dissected soma size: 47.17650646"	"dissected soma size: 44.49592707"	"dissected soma size: 52.63619759"	"dissected soma size: 60.86975553"	"dissected soma size: 57.45876953"	"dissected soma size: 55.54331165"	"dissected soma size: 69.18169939"	"dissected soma size: 46.63360569"	"dissected soma size: 47.32471233"	"dissected soma size: 47.55284923"	"dissected soma size: 45.00804539"	"dissected soma size: 44.30954086"	"dissected soma size: 42.23515873"	"dissected soma size: 41.13557322"	"dissected soma size: 44.07906054"	"dissected soma size: 34.66897966"	"dissected soma size: 48.572759"	"dissected soma size: 39.42873965"	"dissected soma size: 45.43040102"	"dissected soma size: 50.19702284"	"dissected soma size: 46.55162448"	"dissected soma size: 45.47242083"	"dissected soma size: 50.12087055"	"dissected soma size: 47.6998858"	"dissected soma size: 44.4816174"	"dissected soma size: 62.40865191"	"dissected soma size: 77.06920642"	"dissected soma size: 41.84141972"	"dissected soma size: 43.011825"	"dissected soma size: 48.78201256"	"dissected soma size: 70.9803421"	"dissected soma size: 42.20500152"	"dissected soma size: 39.25073089"
cell_identity	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: Glia.cont"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hPropr"	"cell identity: hPEP.NTRK3"	"cell identity: hATF3"	"cell identity: hPEP.KIT"	"cell identity: hPropr"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.0"	"cell identity: hATF3"	"cell identity: hPropr"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hATF3"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPEP.CHRNA7"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: Glia.cont"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: Glia.cont"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: Glia.cont"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPropr"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hATF3"	"cell identity: hATF3"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.SST"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hATF3"	"cell identity: hPEP.KIT"	"cell identity: hPropr"	"cell identity: hPEP.NTRK3"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPEP.SST"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAd.LTMR"	"cell identity: Glia.cont"	"cell identity: hAb.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hNP1"	""	"cell identity: hC.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	""	"cell identity: hAb.LTMR"	""	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPEP.SST"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.SST"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hNP2"	"cell identity: hPEP.PIEZOh"	"cell identity: hATF3"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.SST"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.SST"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hPEP.SST"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.SST"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: hAd.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.SST"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: Glia.cont"	"cell identity: hPEP.0"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: Glia.cont"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: Glia.cont"	"cell identity: hPEP.0"	"cell identity: Glia.cont"	"cell identity: Glia.cont"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP2"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hNP2"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.CHRNA7"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hNP1"	"cell identity: Glia.cont"	"cell identity: Glia.cont"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hPEP.PIEZOh"	"cell identity: hPropr"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hNP1"	"cell identity: hPEP.CHRNA7"	"cell identity: Glia.cont"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: Glia.cont"	"cell identity: hPEP.SST"	"cell identity: hPEP.SST"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hNP1"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hATF3"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.SST"	"cell identity: hPEP.SST"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.SST"	"cell identity: hNP1"	"cell identity: Glia.cont"	"cell identity: hAd.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hNP1"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.CHRNA7"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hAd.LTMR"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hAd.LTMR"	"cell identity: hNP2"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.SST"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: Glia.cont"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: Glia.cont"	"cell identity: hPEP.PIEZOh"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hATF3"	"cell identity: hPEP.SST"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPEP.SST"	"cell identity: Glia.cont"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.SST"	"cell identity: Glia.cont"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hNP2"	"cell identity: Glia.cont"	"cell identity: hAd.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hAd.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	""	"cell identity: hPropr"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: Glia.cont"	"cell identity: hNP1"	"cell identity: hPEP.NTRK3"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hATF3"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hPEP.PIEZOh"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hATF3"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: hATF3"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: hPEP.SST"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAd.LTMR"	"cell identity: hPropr"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hTRPM8"	"cell identity: hC.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.SST"	"cell identity: hPEP.KIT"	"cell identity: hAd.LTMR"	"cell identity: hNP2"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hPEP.SST"	"cell identity: hAb.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hNP1"	"cell identity: hPEP.SST"	"cell identity: hPEP.SST"	"cell identity: hNP2"	"cell identity: hPropr"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.KIT"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.PIEZOh"	"cell identity: Glia.cont"	"cell identity: hPEP.CHRNA7"	"cell identity: Glia.cont"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.KIT"	"cell identity: hPEP.0"	"cell identity: hNP2"	"cell identity: Glia.cont"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hPEP.NTRK3"	"cell identity: Glia.cont"	"cell identity: hATF3"	"cell identity: hPEP.0"	"cell identity: hPEP.KIT"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.CHRNA7"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hPEP.SST"	"cell identity: hPEP.SST"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.KIT"	"cell identity: hAd.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hPEP.PIEZOh"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: hPEP.SST"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP2"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.CHRNA7"	"cell identity: Glia.cont"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: hNP1"	"cell identity: hPEP.KIT"	"cell identity: hATF3"	"cell identity: hNP1"	"cell identity: hTRPM8"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPEP.PIEZOh"	"cell identity: hNP1"	"cell identity: hC.LTMR"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hNP2"	"cell identity: hPropr"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.KIT"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.KIT"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hTRPM8"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hPEP.KIT"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.PIEZOh"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.PIEZOh"	"cell identity: hAd.LTMR"	"cell identity: hATF3"	"cell identity: hNP1"	"cell identity: hNP2"	"cell identity: Glia.cont"	"cell identity: hPEP.KIT"	"cell identity: Glia.cont"	"cell identity: hNP2"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.KIT"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.SST"	"cell identity: hAd.LTMR"	"cell identity: hNP2"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.NTRK3"	"cell identity: hPropr"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.NTRK3"	"cell identity: Glia.cont"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: Glia.cont"	"cell identity: hPEP.SST"	"cell identity: hPEP.PIEZOh"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: Glia.cont"	"cell identity: Glia.cont"	"cell identity: Glia.cont"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hC.LTMR"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.SST"	"cell identity: hPEP.NTRK3"	"cell identity: hNP1"	"cell identity: Glia.cont"	"cell identity: Glia.cont"	"cell identity: Glia.cont"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: Glia.cont"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.SST"	"cell identity: hPEP.SST"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPropr"	"cell identity: hPEP.PIEZOh"	"cell identity: hAb.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.0"	"cell identity: hPEP.SST"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.PIEZOh"	"cell identity: hNP1"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hNP2"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.KIT"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hAd.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPEP.PIEZOh"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hAd.LTMR"	"cell identity: hATF3"	"cell identity: hPropr"	"cell identity: hPEP.SST"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPropr"	"cell identity: hAd.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.KIT"	"cell identity: hNP1"	"cell identity: hNP2"	"cell identity: hPEP.KIT"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAb.LTMR"	"cell identity: hTRPM8"	"cell identity: hAd.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: hNP2"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: hPropr"	"cell identity: hPEP.PIEZOh"	"cell identity: hAd.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hC.LTMR"	"cell identity: hPEP.KIT"	"cell identity: hNP1"	"cell identity: Glia.cont"	"cell identity: hPropr"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: Glia.cont"	"cell identity: Glia.cont"	"cell identity: hPEP.0"	"cell identity: hPEP.CHRNA7"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hPEP.KIT"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAb.LTMR"	"cell identity: hATF3"	"cell identity: hPEP.SST"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPropr"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hPEP.KIT"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hTRPM8"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hPEP.SST"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hPEP.SST"	"cell identity: hPropr"	"cell identity: hAd.LTMR"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hNP1"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.NTRK3"	"cell identity: Glia.cont"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hAd.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hNP2"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hAd.LTMR"	"cell identity: hNP2"	"cell identity: hPropr"	"cell identity: hATF3"	"cell identity: hPropr"	"cell identity: hPEP.KIT"	"cell identity: hPEP.SST"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.TRPV1/A1.1"	""	"cell identity: hNP1"	"cell identity: hPEP.SST"	"cell identity: hPEP.SST"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.PIEZOh"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.0"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.SST"	"cell identity: hPEP.SST"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPEP.KIT"	"cell identity: hAb.LTMR"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hTRPM8"	"cell identity: hPEP.0"	"cell identity: hPEP.KIT"	"cell identity: hTRPM8"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hATF3"	"cell identity: hPEP.PIEZOh"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hAd.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: Glia.cont"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.0"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.0"	"cell identity: Glia.cont"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.PIEZOh"	"cell identity: hAb.LTMR"	"cell identity: hATF3"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hPEP.KIT"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: Glia.cont"	"cell identity: hPEP.PIEZOh"	"cell identity: hPropr"	"cell identity: hAd.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.0"	""	"cell identity: hPEP.0"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.PIEZOh"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hATF3"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hPEP.NTRK3"	"cell identity: hNP2"	"cell identity: Glia.cont"	"cell identity: hAb.LTMR"	"cell identity: hC.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hC.LTMR"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hPEP.SST"	"cell identity: hNP1"	"cell identity: hPropr"	"cell identity: hPEP.KIT"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.CHRNA7"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hATF3"	"cell identity: hATF3"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: Glia.cont"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hPropr"	"cell identity: hPEP.CHRNA7"	"cell identity: hC.LTMR"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: Glia.cont"	"cell identity: hAb.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPEP.CHRNA7"	"cell identity: hPEP.NTRK3"	"cell identity: hPEP.0"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.0"	"cell identity: hPEP.NTRK3"	"cell identity: hAb.LTMR"	"cell identity: hPropr"	"cell identity: hAb.LTMR"	"cell identity: hPEP.0"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hC.LTMR"	"cell identity: hPropr"	"cell identity: Glia.cont"	"cell identity: hNP1"	"cell identity: hPEP.TRPV1/A1.2"	"cell identity: hPEP.NTRK3"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hAb.LTMR"	"cell identity: hPEP.KIT"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPEP.0"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPEP.0"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hPEP.TRPV1/A1.1"	"cell identity: hPropr"	"cell identity: hNP1"	"cell identity: hAd.LTMR"	"cell identity: hNP1"	"cell identity: hNP1"	"cell identity: hPEP.0"	"cell identity: hPEP.0"	"cell identity: hTRPM8"	"cell identity: hPEP.PIEZOh"	"cell identity: hAd.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hNP1"	"cell identity: hPEP.PIEZOh"	"cell identity: hAb.LTMR"	"cell identity: Glia.cont"	"cell identity: hAd.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hAb.LTMR"	"cell identity: hAd.LTMR"	"cell identity: hPEP.0"	"cell identity: hPropr"	"cell identity: hPropr"	"cell identity: hPEP.PIEZOh"	"cell identity: hPEP.SST"
"""

meta_df = pd.read_csv(io.StringIO(meta), header=None, sep="\t")
meta_df = meta_df.T
meta_df.columns = meta_df.iloc[0]
meta_df  = meta_df[1:]
for c in meta_df.columns:
    meta_df[c] = [x.split(": ")[-1] if x not in [None, np.nan] else x for x in meta_df[c]]

meta_df.index = list(meta_df["sample_id"]).copy()
meta_df

In [ ]:
adata.obs = adata.obs.merge(meta_df, how="left", left_index=True, right_index=True)
adata.obs

In [ ]:
# process dataset
ensure_unique_idx(adata)
enforce_sparsity(adata)
fix_nan_x(adata)
filter_nan_var_names(adata)
del_raw(adata)
calc_n_genes(adata)
filter_low_quality_cells(adata)
normalize_total(adata, target_sum=1e4)
adata

In [ ]:
adata.var

In [ ]:
fname = os.path.join(source, f"{dataset}.h5ad")
print(fname)
adata.write(fname, compression="gzip")